In [1]:
import os
import pandas as pd
import numpy as np
import re
from datetime import date, datetime, timedelta
import warnings
import glob 
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows

# Desactiva las advertencias
warnings.filterwarnings('ignore')

In [2]:
# Definir las columnas de códigos y de diagnósticos en texto libre
columnas_codigos = ['COD_DIAGNOSTICO1', 'COD_DIAGNOSTICO2', 'COD_DIAGNOSTICO3',
                    'COD_DIAGNOSTICO4', 'COD_DIAGNOSTICO5', 'COD_DIAGNOSTICO6',
                    'COD_DIAGNOSTICO7', 'COD_DIAGNOSTICO8', 'COD_DIAGNOSTICO9',
                    'COD_DIAGNOSTICO10']
columnas_diagnosticos = ['DIAGNOSTICO1', 'DIAGNOSTICO2', 'DIAGNOSTICO3',
                         'DIAGNOSTICO4', 'DIAGNOSTICO5', 'DIAGNOSTICO6',
                         'DIAGNOSTICO7', 'DIAGNOSTICO8', 'DIAGNOSTICO9',
                         'DIAGNOSTICO10']


In [3]:
def cargar_dataframe_ambulatorio(ruta):
    encoding = 'ISO-8859-1'
    delimitador = ';'
    return pd.read_csv(ruta, delimiter=delimitador, encoding=encoding, on_bad_lines='skip')

def cargar_dataframe_ieeh(ruta):
    encoding = 'ISO-8859-1'
    delimitador = ';'
    return pd.read_csv(ruta, delimiter=delimitador, encoding=encoding, on_bad_lines='skip')

def filtro_dataframe_ca_tiroides(dataframe):

    # Se crea un patrón regex para identificar los códigos CIE-! asociados al cáncer de tiroides
    patron_cancer_tiroides = r'^C73'

    dataframe[columnas_codigos] = dataframe[columnas_codigos].astype(str)
    dataframe[columnas_diagnosticos] = dataframe[columnas_diagnosticos].astype(str)
    # Filtrar el DataFrame para encontrar las filas que contienen el código de cáncer de tiroides
    filtro_codigos = dataframe[columnas_codigos].apply(lambda x: x.str.contains(patron_cancer_tiroides, na=False)).any(axis=1)

    # Patrones para identificar diagnósticos de cáncer de tiroides en texto libre
    #patron_diagnostico_tiroides = re.compile(r'\bc(?!o)[a-zá-ú]*[ \.]*tiroide(?!o)[a-zá-ú]*\b', re.IGNORECASE)
    patron_diagnostico_tiroides = re.compile(r'\bc(?!oe).*tiroide(?!o)[a-zá-ú]*\b', re.IGNORECASE)



    patron_carci_in_situ = re.compile(r'carci.*tiroid[a-zá-ú]*', re.IGNORECASE)
    patron_tumor_maligno = re.compile(r'tumor.*maligno.*glandula tiroides|tumor.*maligno.*tiroid[a-zá-ú]*', re.IGNORECASE)
    patron_carci_in_situ_tiroides = re.compile(r'carci.*in\s*situ.*tiroid[a-zá-ú]*', re.IGNORECASE)

    patron_nodulos_tirad_4_5 = re.compile(r'n[oó]dulos.*tirad\s*[5]', re.IGNORECASE)# sacar 4

    # Patrón para identificar nódulo tiroideo solitario no tóxico
    #patron_nodulo_tiroideo_solitario = re.compile(r'n[oó]dulo.*tiroideo.*solitario.*no.*t[oó]xico', re.IGNORECASE)#sacar

    # Filtrar el DataFrame para encontrar las filas que contienen el diagnóstico de cáncer de tiroides en texto libre
    filtro_diagnosticos = dataframe[columnas_diagnosticos].apply(lambda x:
        x.str.contains(patron_diagnostico_tiroides, na=False) |
        x.str.contains(patron_carci_in_situ, na=False) |
        x.str.contains(patron_tumor_maligno, na=False) |
        x.str.contains(patron_carci_in_situ_tiroides, na=False) |
        x.str.contains(patron_nodulos_tirad_4_5, na=False)).any(axis=1)

    # Combinar los filtros para obtener las filas que cumplen con al menos uno de los criterios
    return dataframe[filtro_codigos | filtro_diagnosticos].copy()

def filtro_dataframe_ca_tiroides_ieeh(dataframe):

    # Definir las columnas de códigos y de diagnósticos en texto libre

    columnas_diagnosticos = ['DIAG1','DIAG2','DIAG3']
    # Llenar los valores NaN con una cadena vacía en las columnas de diagnóstico
    dataframe[columnas_diagnosticos] = dataframe[columnas_diagnosticos].fillna('')

    dataframe[columnas_diagnosticos] = dataframe[columnas_diagnosticos].astype(str)


    #patron_diagnostico_tiroides = re.compile(r'\bc[a-zá-ú]*[ \.]*tiroide[a-zá-ú]*\b', re.IGNORECASE)
    patron_diagnostico_tiroides = re.compile(r'\bc(?!oe).*tiroide(?!o)[a-zá-ú]*\b', re.IGNORECASE)


    #patron_diagnostico_tiroides = re.compile(r'c[a-zá-ú]*[ \.]*tiroid[a-zá-ú]*', re.IGNORECASE)
    patron_carci_in_situ = re.compile(r'carci.*tiroid[a-zá-ú]*', re.IGNORECASE)
    patron_tumor_maligno = re.compile(r'tumor.*maligno.*glandula tiroides|tumor.*maligno.*tiroid[a-zá-ú]*', re.IGNORECASE)
    # Patrones para identificar 'carcinoma in situ de glándula tiroides' de forma más general
    patron_carci_in_situ_tiroides = re.compile(r'carci.*in\s*situ.*tiroid[a-zá-ú]*', re.IGNORECASE)

    # Patrones para identificar 'nódulos tiroideos tirad 4 y tirad 5' de forma más general
    patron_nodulos_tirad_4_5 = re.compile(r'n[oó]dulos.*tirad\s*[5]', re.IGNORECASE)# sacar 4

    # Filtrar el DataFrame para encontrar las filas que contienen el diagnóstico de cáncer de tiroides en texto libre
    filtro_diagnosticos = dataframe[columnas_diagnosticos].apply(lambda x:
        x.str.contains(patron_diagnostico_tiroides, na=False) |
        x.str.contains(patron_carci_in_situ, na=False) |
        x.str.contains(patron_tumor_maligno, na=False) |
        x.str.contains(patron_carci_in_situ_tiroides, na=False) |
        x.str.contains(patron_nodulos_tirad_4_5, na=False)).any(axis=1)

    # Combinar los filtros para obtener las filas que cumplen con al menos uno de los criterios
    resultado_filtrado = dataframe[filtro_diagnosticos].copy()
    return resultado_filtrado

def filtro_dataframe_ca_tiroides_urgencias(dataframe):
    # Definir las columnas de códigos y de diagnósticos en texto libre
    columnas_codigos = ['CODDIAGNOSTICO']
    columnas_diagnosticos = ['DESCDIAGNOSTICO']

    # Se crea un patrón regex para identificar los códigos CIE-! asociados al cáncer de tiroides
    patron_cancer_tiroides = r'^C73'

    dataframe[columnas_codigos] = dataframe[columnas_codigos].astype(str)
    dataframe[columnas_diagnosticos] = dataframe[columnas_diagnosticos].astype(str)

    # Filtrar el DataFrame para encontrar las filas que contienen el código de cáncer de tiroides
    filtro_codigos = dataframe[columnas_codigos].apply(lambda x: x.str.contains(patron_cancer_tiroides, na=False)).any(axis=1)

    # Patrones para identificar diagnósticos de cáncer de tiroides en texto libre
    #patron_diagnostico_tiroides = re.compile(r'c[a-zá-ú]*[ \.]*tiroid[a-zá-ú]*', re.IGNORECASE)
    #patron_diagnostico_tiroides = re.compile(r'\bc[a-zá-ú]*[ \.]*tiroide[a-zá-ú]*\b', re.IGNORECASE)
    patron_diagnostico_tiroides = re.compile(r'\bc(?!oe).*tiroide(?!o)[a-zá-ú]*\b', re.IGNORECASE)


    patron_carci_in_situ = re.compile(r'carci.*tiroid[a-zá-ú]*', re.IGNORECASE)
    patron_tumor_maligno = re.compile(r'tumor.*maligno.*glandula tiroides|tumor.*maligno.*tiroid[a-zá-ú]*', re.IGNORECASE)
    # Patrones para identificar 'carcinoma in situ de glándula tiroides' de forma más general
    patron_carci_in_situ_tiroides = re.compile(r'carci.*in\s*situ.*tiroid[a-zá-ú]*', re.IGNORECASE)

    # Patrones para identificar 'nódulos tiroideos tirad 4 y tirad 5' de forma más general
    patron_nodulos_tirad_4_5 = re.compile(r'n[oó]dulos.*tirad\s*[5]', re.IGNORECASE)# sacar 4

    # Patrón para identificar nódulo tiroideo solitario no tóxico
    #patron_nodulo_tiroideo_solitario = re.compile(r'n[oó]dulo.*tiroideo.*solitario.*no.*t[oó]xico', re.IGNORECASE)#sacar

    # Filtrar el DataFrame para encontrar las filas que contienen el diagnóstico de cáncer de tiroides en texto libre
    filtro_diagnosticos = dataframe[columnas_diagnosticos].apply(lambda x:
        x.str.contains(patron_diagnostico_tiroides, na=False) |
        x.str.contains(patron_carci_in_situ, na=False) |
        x.str.contains(patron_tumor_maligno, na=False) |
        x.str.contains(patron_carci_in_situ_tiroides, na=False) |
        x.str.contains(patron_nodulos_tirad_4_5, na=False)).any(axis=1)

    # Combinar los filtros para obtener las filas que cumplen con al menos uno de los criterios
    resultado_filtrado = dataframe[filtro_codigos | filtro_diagnosticos].copy()
    return resultado_filtrado

def filtro_dataframe_ca_tiroides_inter(dataframe):
    # Definir las columnas de códigos y de diagnósticos en texto libre
    columnas_codigos = ['SOSP_DIAG_COD_1', 'SOSP_DIAG_COD_2', 'SOSP_DIAG_COD_3', 'SOSP_DIAG_COD_4', 'SOSP_DIAG_COD_5']
    columnas_diagnosticos = ['SOSP_DIAG_DESC_1', 'SOSP_DIAG_DESC_2', 'SOSP_DIAG_DESC_3', 'SOSP_DIAG_DESC_4', 'SOSP_DIAG_DESC_5']

    # Se crea un patrón regex para identificar los códigos CIE-! asociados al cáncer de tiroides
    patron_cancer_tiroides = r'^C73'

    dataframe[columnas_codigos] = dataframe[columnas_codigos].astype(str)
    dataframe[columnas_diagnosticos] = dataframe[columnas_diagnosticos].astype(str)

    # Filtrar el DataFrame para encontrar las filas que contienen el código de cáncer de tiroides
    filtro_codigos = dataframe[columnas_codigos].apply(lambda x: x.str.contains(patron_cancer_tiroides, na=False)).any(axis=1)

    # Patrones para identificar diagnósticos de cáncer de tiroides en texto libre
    #patron_diagnostico_tiroides = re.compile(r'\bc[a-zá-ú]*[ \.]*tiroide[a-zá-ú]*\b', re.IGNORECASE)
    patron_diagnostico_tiroides = re.compile(r'\bc(?!oe).*tiroide(?!o)[a-zá-ú]*\b', re.IGNORECASE)

    patron_carci_in_situ = re.compile(r'carci.*tiroid[a-zá-ú]*', re.IGNORECASE)
    patron_tumor_maligno = re.compile(r'tumor.*maligno.*glandula tiroides|tumor.*maligno.*tiroid[a-zá-ú]*', re.IGNORECASE)
    patron_carci_in_situ_tiroides = re.compile(r'carci.*in\s*situ.*tiroid[a-zá-ú]*', re.IGNORECASE)
    patron_nodulos_tirad_4_5 = re.compile(r'n[oó]dulos.*tirad\s*[5]', re.IGNORECASE)

    # Filtrar el DataFrame para encontrar las filas que contienen el diagnóstico de cáncer de tiroides en texto libre
    filtro_diagnosticos = dataframe[columnas_diagnosticos].apply(lambda x:
        x.str.contains(patron_diagnostico_tiroides, na=False) |
        x.str.contains(patron_carci_in_situ, na=False) |
        x.str.contains(patron_tumor_maligno, na=False) |
        x.str.contains(patron_carci_in_situ_tiroides, na=False) |
        x.str.contains(patron_nodulos_tirad_4_5, na=False)).any(axis=1)

    # Filtrar por la comuna La Florida
    filtro_comuna = dataframe['COMUNA'] == 'La Florida'

    # Combinar los filtros para obtener las filas que cumplen con al menos uno de los criterios
    resultado_filtrado = dataframe[(filtro_codigos | filtro_diagnosticos) ].copy()
    return resultado_filtrado

def agregar_dv(resultado_filtrado):
    # Verificar si el tipo de identificador es "RUN" y quitar el último dígito de NRO_IDENTIFICADOR_PACIENTE
    #condicion = (resultado_filtrado['TIPO_IDENTIFICADOR_PACIENTE'] == 'RUN')
    # Crear una nueva columna 'DV' que contenga el último dígito de NRO_IDENTIFICADOR_PACIENTE
    resultado_filtrado['DV'] = resultado_filtrado['NRO_IDENTIFICADOR_PACIENTE'].str[-1]

    #resultado_filtrado.loc[condicion, 'NRO_IDENTIFICADOR_PACIENTE'] = resultado_filtrado.loc[condicion, 'NRO_IDENTIFICADOR_PACIENTE'].str[:-1]
    resultado_filtrado['NRO_IDENTIFICADOR_PACIENTE'] = resultado_filtrado['NRO_IDENTIFICADOR_PACIENTE'].str[:-1]

    return resultado_filtrado

def agregar_dv_urgencias(resultado_filtrado):

    # Verificar si el tipo de identificador es "RUN" y quitar el último dígito de NRO_IDENTIFICADOR_PACIENTE
    # Crear una nueva columna 'DV' que contenga el último dígito de NRO_IDENTIFICADOR_PACIENTE
    resultado_filtrado['DV'] = resultado_filtrado['RUT'].str[-1]

    resultado_filtrado['RUT'] = resultado_filtrado['RUT'].str[:-1]

    return resultado_filtrado

def agregar_dv_inter(resultado_filtrado):
    # Verificar si el tipo de identificador es "RUN" y quitar el último dígito de NRO_IDENTIFICADOR_PACIENTE
    # Crear una nueva columna 'DV' que contenga el último dígito de NRO_IDENTIFICADOR_PACIENTE

    resultado_filtrado['DV'] = resultado_filtrado['NRO_IDENTIFICADOR'].str[-1]

    resultado_filtrado['NRO_IDENTIFICADOR'] = resultado_filtrado['NRO_IDENTIFICADOR'].str[:-1]

    return resultado_filtrado

def obtener_diagnosticos_y_codigos(df, nro_identificacion):
    df_paciente = df[df['NRO_IDENTIFICADOR_PACIENTE'] == nro_identificacion]
    diagnosticos = df_paciente[['DIAGNOSTICO1', 'DIAGNOSTICO2', 'DIAGNOSTICO3', 'DIAGNOSTICO4',
                                'DIAGNOSTICO5', 'DIAGNOSTICO6', 'DIAGNOSTICO7','DIAGNOSTICO8',
                                'DIAGNOSTICO9','DIAGNOSTICO10']].values.flatten()
    codigos_diagnostico = df_paciente[['COD_DIAGNOSTICO1', 'COD_DIAGNOSTICO2', 'COD_DIAGNOSTICO3', 'COD_DIAGNOSTICO4',
                                       'COD_DIAGNOSTICO5', 'COD_DIAGNOSTICO6', 'COD_DIAGNOSTICO7','COD_DIAGNOSTICO8',
                                       'COD_DIAGNOSTICO9','COD_DIAGNOSTICO10']].values.flatten()

    diagnosticos = [diagnostico for diagnostico in diagnosticos if pd.notnull(diagnostico)]
    codigos_diagnostico = [codigo for codigo in codigos_diagnostico if pd.notnull(codigo)]

    return list(set(diagnosticos)), list(set(codigos_diagnostico))

def obtener_diagnosticos_ieeh(df, nro_identificacion):
    df_paciente = df[df['RUT'] == nro_identificacion]
    diagnosticos = df_paciente[['DIAG1','DIAG2','DIAG3']].values.flatten()
    diagnosticos = [diagnostico for diagnostico in diagnosticos if pd.notnull(diagnostico)]

    return list(set(diagnosticos))

def obtener_diagnosticos_urgencias(df, nro_identificacion):
    df_paciente = df[df['RUT'] == nro_identificacion]
    diagnosticos = df_paciente[['DESCDIAGNOSTICO']].values.flatten()
    codigos_diagnostico = df_paciente[['CODDIAGNOSTICO']].values.flatten()

    diagnosticos = [diagnostico for diagnostico in diagnosticos if pd.notnull(diagnostico)]
    codigos_diagnostico = [codigo for codigo in codigos_diagnostico if pd.notnull(codigo)]

    return list(set(diagnosticos)), list(set(codigos_diagnostico))

def obtener_diagnosticos_inter(df, nro_identificacion):
    df_paciente = df[df['NRO_IDENTIFICADOR'] == nro_identificacion]
    diagnosticos = df_paciente[['SOSP_DIAG_DESC_1', 'SOSP_DIAG_DESC_2', 'SOSP_DIAG_DESC_3', 'SOSP_DIAG_DESC_4', 'SOSP_DIAG_DESC_5']].values.flatten()
    codigos_diagnostico = df_paciente[['SOSP_DIAG_COD_1', 'SOSP_DIAG_COD_2', 'SOSP_DIAG_COD_3', 'SOSP_DIAG_COD_4', 'SOSP_DIAG_COD_5']].values.flatten()

    diagnosticos = [diagnostico for diagnostico in diagnosticos if pd.notnull(diagnostico)]
    codigos_diagnostico = [codigo for codigo in codigos_diagnostico if pd.notnull(codigo)]

    return list(set(diagnosticos)), list(set(codigos_diagnostico))

def agregar_diagnosticos_y_codigos(resultado_filtrado):
    if resultado_filtrado.empty:
        return pd.DataFrame(columns=['RUT','DIAGNOSTICOS', 'CODIGOS_DIAGNOSTICO'])

    # Aplicar la función de agregación en el DataFrame
    resultados = resultado_filtrado.groupby('NRO_IDENTIFICADOR_PACIENTE').agg({'NRO_IDENTIFICADOR_PACIENTE': 'first', 'DIAGNOSTICO1': 'first',
                                                            'DIAGNOSTICO2': 'first','DIAGNOSTICO3': 'first','DIAGNOSTICO4': 'first',
                                                            'DIAGNOSTICO5': 'first','DIAGNOSTICO6': 'first','DIAGNOSTICO7': 'first',
                                                            'DIAGNOSTICO8': 'first','DIAGNOSTICO9': 'first','DIAGNOSTICO10': 'first',
                                                            'COD_DIAGNOSTICO1': 'first','COD_DIAGNOSTICO2': 'first', 'COD_DIAGNOSTICO2': 'first',
                                                            'COD_DIAGNOSTICO3': 'first', 'COD_DIAGNOSTICO4': 'first', 'COD_DIAGNOSTICO5': 'first',
                                                            'COD_DIAGNOSTICO6': 'first', 'COD_DIAGNOSTICO7': 'first','COD_DIAGNOSTICO8': 'first',
                                                                'COD_DIAGNOSTICO8': 'first',  'COD_DIAGNOSTICO9': 'first',
                                                                'COD_DIAGNOSTICO10': 'first'}).reset_index(drop=True)
    resultados['DIAGNOSTICOS'], resultados['CODIGOS_DIAGNOSTICO'] = zip(*resultados['NRO_IDENTIFICADOR_PACIENTE'].apply(lambda x: obtener_diagnosticos_y_codigos(resultado_filtrado, x)))
    # Renombrar la columna 'old_name' a 'new_name'
    resultados = resultados.rename(columns={'NRO_IDENTIFICADOR_PACIENTE': 'RUT'})
    return resultados

def agregar_diagnosticos_ieeh(resultado_filtrado):
    if resultado_filtrado.empty:
        return pd.DataFrame(columns=['RUT', 'DESCDIAGNOSTICO', 'CODDIAGNOSTICO', 'DIAGNOSTICOS', 'CODIGOS_DIAGNOSTICO'])

    # Aplicar la función de agregación en el DataFrame
    resultados = resultado_filtrado.groupby('RUT').agg({'RUT': 'first', 'DIAG1': 'first',
                                                            'DIAG1': 'first','DIAG3': 'first'}).reset_index(drop=True)
    #resultados['CODIGOS_DIAGNOSTICO'] = []
    resultados['DIAGNOSTICOS'] = resultados['RUT'].apply(lambda x: obtener_diagnosticos_ieeh(resultado_filtrado, x))
    # Renombrar la columna 'old_name' a 'new_name'
    resultados = resultados.rename(columns={'RUT': 'RUT'})
    resultados['CODIGOS_DIAGNOSTICO'] = np.nan
    return resultados

def agregar_diagnosticos_y_codigos_urgencias(resultado_filtrado):
    if resultado_filtrado.empty:
        return pd.DataFrame(columns=['RUT', 'DESCDIAGNOSTICO', 'CODDIAGNOSTICO', 'DIAGNOSTICOS', 'CODIGOS_DIAGNOSTICO'])

    resultados = resultado_filtrado.groupby('RUT').agg({'RUT': 'first', 'DESCDIAGNOSTICO': 'first',
                                                        'CODDIAGNOSTICO': 'first'}).reset_index(drop=True)
    resultados['DIAGNOSTICOS'], resultados['CODIGOS_DIAGNOSTICO'] = zip(*resultados['RUT'].apply(lambda x: obtener_diagnosticos_urgencias(resultado_filtrado, x)))
    # Renombrar la columna 'old_name' a 'new_name'
    resultados = resultados.rename(columns={'RUT': 'RUT'})

    return resultados

def agregar_diagnosticos_y_codigos_inter(resultado_filtrado):
    if resultado_filtrado.empty:
        return pd.DataFrame(columns=['RUT','DIAGNOSTICOS', 'CODIGOS_DIAGNOSTICO'])

    # Aplicar la función de agregación en el DataFrame
    resultados = resultado_filtrado.groupby('NRO_IDENTIFICADOR').agg({'NRO_IDENTIFICADOR': 'first', 'SOSP_DIAG_DESC_1': 'first',
                                                            'SOSP_DIAG_DESC_2': 'first','SOSP_DIAG_DESC_3': 'first','SOSP_DIAG_DESC_4': 'first',
                                                            'SOSP_DIAG_DESC_5': 'first','SOSP_DIAG_COD_1': 'first','SOSP_DIAG_COD_2': 'first',
                                                                      'SOSP_DIAG_COD_3': 'first',
                                                            'SOSP_DIAG_COD_4': 'first', 'SOSP_DIAG_COD_5': 'first'
                                                            }).reset_index(drop=True)
    resultados['DIAGNOSTICOS'], resultados['CODIGOS_DIAGNOSTICO'] = zip(*resultados['NRO_IDENTIFICADOR'].apply(lambda x: obtener_diagnosticos_inter(resultado_filtrado, x)))
    # Renombrar la columna 'old_name' a 'new_name'
    resultados = resultados.rename(columns={'NRO_IDENTIFICADOR': 'RUT'})
    return resultados

def crear_dataframe_ambulatorio(resultado_filtrado):
    nuevo_dataframe = pd.DataFrame({
        'Problema de Salud': 'Cancer de Tiroides',
        'RUT': resultado_filtrado['NRO_IDENTIFICADOR_PACIENTE'],
        'DV': resultado_filtrado['DV'],
        'Nombre del Paciente':resultado_filtrado['PACIENTE'],
        
        'N° Telefono 1': resultado_filtrado.get('TELEFONO_MOVIL', np.nan),
        'N° Telefono 2':resultado_filtrado.get('TELEFONO_FIJO_1', np.nan),
        'N° Telefono 3': resultado_filtrado.get('TELEFONO_FIJO_2', np.nan),
        'Fecha Proximo Hito': np.nan,
        #'Observaciones del Diagnostico': np.nan,
        #'Diagnostico CIE-10': np.nan,
        'Fecha Proximo Hito': np.nan,
        'Nombre agenda Medico especialista': np.nan,
    })
   
    return nuevo_dataframe

def crear_dataframe_ieeh(resultado_filtrado):
    nuevo_dataframe = pd.DataFrame({
        'Problema de Salud': 'Cancer de Tiroides',
        'RUT': resultado_filtrado['RUT'],
        'DV': resultado_filtrado['DV'],
        'Nombre del Paciente':resultado_filtrado['NOMBRES']+ ' '+ resultado_filtrado['APELL_PATE']+ ' '+ resultado_filtrado['APELL_MATE'] ,
        'N° Telefono 1': resultado_filtrado['TELEFONO'],
        'N° Telefono 2': np.nan,
        'N° Telefono 3': np.nan,
        'Fecha Proximo Hito': np.nan,
        #'Observaciones del Diagnostico': np.nan,
        #'Diagnostico CIE-10': np.nan,
        'Fecha Proximo Hito': np.nan,
        'Nombre agenda Medico especialista': np.nan,
    })

    return nuevo_dataframe

def crear_dataframe_urgencias(resultado_filtrado):
    nuevo_dataframe = pd.DataFrame({
        'Problema de Salud': 'Cancer de Tiroides',
        'RUT': resultado_filtrado['RUT'],
        'DV': resultado_filtrado['DV'],
        'Nombre del Paciente': resultado_filtrado['NOMBRES']+ ' ' + resultado_filtrado['APELL_PATE']+ ' '+ resultado_filtrado['APELL_MATE'] ,
        'N° Telefono 1': resultado_filtrado['TELEFONO'],
        'N° Telefono 2': np.nan,
        'N° Telefono 3': np.nan,
        'Fecha Proximo Hito': np.nan,
        #'Observaciones del Diagnostico': np.nan,
        #'Diagnostico CIE-10': np.nan,
        'Fecha Proximo Hito': np.nan,
        'Nombre agenda Medico especialista': np.nan,
    })

    return nuevo_dataframe

def crear_dataframe_urgencias(resultado_filtrado):
    nuevo_dataframe = pd.DataFrame({
        'Problema de Salud': 'Cancer de Tiroides',
        'RUT': resultado_filtrado['RUT'],
        'DV': resultado_filtrado['DV'],
        'Nombre del Paciente':resultado_filtrado['NOMBRES_PACIENTE'] + ' ' + resultado_filtrado['APELLIDOS_PACIENTE'],
        'N° Telefono 1': np.nan,
        'N° Telefono 2': np.nan,
        'N° Telefono 3': np.nan,
        'Fecha Proximo Hito': np.nan,
        #'Observaciones del Diagnostico': np.nan,
        #'Diagnostico CIE-10': np.nan,
        'Fecha Proximo Hito': np.nan,
        'Nombre agenda Medico especialista': np.nan,
    })

    return nuevo_dataframe

def crear_dataframe_inter(resultado_filtrado):
    nuevo_dataframe = pd.DataFrame({
        'Problema de Salud': 'Cancer de Tiroides',
        'RUT': resultado_filtrado['NRO_IDENTIFICADOR'],
        'DV': resultado_filtrado['DV'],
        'Nombre del Paciente':resultado_filtrado['NOM_PAC'] + ' ' + resultado_filtrado['APELLPAT_PAC']+' '+resultado_filtrado['APELLMAT_PAC'],
        'N° Telefono 1': resultado_filtrado['FONO_MOVIL'],
        'N° Telefono 2': resultado_filtrado['FONO_FIJO_1'],
        'N° Telefono 3': resultado_filtrado['FONO_FIJO_2'],
        'Fecha Proximo Hito': np.nan,
        #'Observaciones del Diagnostico': np.nan,
        #'Diagnostico CIE-10': np.nan,
        'Fecha Proximo Hito': np.nan,
        'Nombre agenda Medico especialista': np.nan,
    })

    return nuevo_dataframe

def safe_convert_to_int(value):
    try:
        return int(value)
    except ValueError:
        return value

def merge_ambulatorio(resultados, nuevo_dataframe):
    df1 = resultados[['RUT','DIAGNOSTICOS', 'CODIGOS_DIAGNOSTICO']]
    df2 = nuevo_dataframe

    # Realizar merge por la columna 'RUT'
    df1['RUT'] = df1['RUT'].astype(str)
    df2['RUT'] = df2['RUT'].astype(str)

    df1['RUT'] = df1['RUT'].apply(safe_convert_to_int)
    df2['RUT'] = df2['RUT'].apply(safe_convert_to_int)

    # Realizar merge por la columna 'RUT'
    merged_df = pd.merge(df1, df2, on='RUT', how='outer')

    # El parámetro 'on' especifica la columna por la cual se realiza el merge
    # El parámetro 'how' especifica el tipo de merge. 'inner' conserva solo las filas que tienen un RUT que está presente en ambos DataFrames.
    
    return merged_df 

def merge_urgencia(resultados, nuevo_dataframe):
    df1 = resultados[['RUT','DIAGNOSTICOS', 'CODIGOS_DIAGNOSTICO']]
    df2 = nuevo_dataframe

    # Realizar merge por la columna 'RUT'
    df1['RUT'] = df1['RUT'].astype(str)
    df2['RUT'] = df2['RUT'].astype(str)

    df1['RUT'] = df1['RUT'].apply(safe_convert_to_int)
    df2['RUT'] = df2['RUT'].apply(safe_convert_to_int)

    # Realizar merge por la columna 'RUT'
    merged_df = pd.merge(df1, df2, on='RUT', how='outer')

    # El parámetro 'on' especifica la columna por la cual se realiza el merge
    # El parámetro 'how' especifica el tipo de merge. 'inner' conserva solo las filas que tienen un RUT que está presente en ambos DataFrames.

    return merged_df

def merge_inter(resultados, nuevo_dataframe):
    df1 = resultados[['RUT','DIAGNOSTICOS', 'CODIGOS_DIAGNOSTICO']]
    df2 = nuevo_dataframe

    # Realizar merge por la columna 'RUT'
    df1['RUT'] = df1['RUT'].astype(str)
    df2['RUT'] = df2['RUT'].astype(str)

    df1['RUT'] = df1['RUT'].apply(safe_convert_to_int)
    df2['RUT'] = df2['RUT'].apply(safe_convert_to_int)

    # Realizar merge por la columna 'RUT'
    merged_df = pd.merge(df1, df2, on='RUT', how='outer')

    # El parámetro 'on' especifica la columna por la cual se realiza el merge
    # El parámetro 'how' especifica el tipo de merge. 'inner' conserva solo las filas que tienen un RUT que está presente en ambos DataFrames.

    return merged_df

def filtrar_columnas_relevantes(merged_df):
    nuevo_dataframe = merged_df.drop_duplicates(subset='RUT', keep='first')
    nuevo_dataframe = nuevo_dataframe[['Problema de Salud','RUT' ,'DV',
       'Nombre del Paciente', 'N° Telefono 1', 'N° Telefono 2',
       'N° Telefono 3', 'Fecha Proximo Hito', 'DIAGNOSTICOS', 'CODIGOS_DIAGNOSTICO',
       'Nombre agenda Medico especialista']]
    nuevo_dataframe = nuevo_dataframe.rename(columns={'Problema de Salud': 'Problema Oncologico'})
    nuevo_dataframe['Nombre garantia'] = np.nan
    nuevo_dataframe= nuevo_dataframe[['Problema Oncologico',
    'RUT',
    'DV',
    'Nombre del Paciente',
    'N° Telefono 1',
    'N° Telefono 2',
    'N° Telefono 3',
    'Nombre garantia',
    'Fecha Proximo Hito',
    'DIAGNOSTICOS',
    'CODIGOS_DIAGNOSTICO',
    'Nombre agenda Medico especialista']]
    return nuevo_dataframe

def crear_dataframe_ges(dataframe_tiroides):
    # Crear un nuevo DataFrame con las columnas deseadas
    nuevo_dataframe_2 = pd.DataFrame({
        'Problema Oncologico': dataframe_tiroides['Problema de salud'],
        'RUT': dataframe_tiroides['RUT'],
        'DV': dataframe_tiroides['DV'],
        'Nombre del Paciente':dataframe_tiroides['Nombre'],
        'N° Telefono 1': dataframe_tiroides['Telefonos.x'],
        'N° Telefono 2': dataframe_tiroides['Telefonos.y'],
        'N° Telefono 3': np.nan,
        'Nombre garantia': dataframe_tiroides['Nombre garantia'],
        'Fecha Proximo Hito': np.nan,
        'DIAGNOSTICOS': np.nan,
        'CODIGOS_DIAGNOSTICO': np.nan,
        'Fecha Proximo Hito': np.nan,
        'Nombre agenda Medico especialista': np.nan,
    })
    return nuevo_dataframe_2

def crear_dataframe_ultima_atencion(dataframe_tiroides):
    # Crear un nuevo DataFrame con las columnas deseadas
    nuevo_dataframe_2 = pd.DataFrame({
        'RUT': dataframe_tiroides['RUT'],
        'Fecha Ultima atencion': dataframe_tiroides['HORA_TERMINO_ATENCION'],
        'Profesional Ultima atencion': dataframe_tiroides['ESTAMENTO_PROFESIONAL_ATENCION']+ ' '+
        dataframe_tiroides['NOMBRE_PROFESIONAL_ATENCION']+ ' ' + dataframe_tiroides['DESCRIPCION'] + ' '+
        dataframe_tiroides['ESPECIALIDAD_AGENDA']
    })
    return nuevo_dataframe_2

def proceso_dataframe(rutas_archivos):
    resultados_totales = []  # Lista para almacenar los resultados de todos los archivos
    for ruta_archivo in rutas_archivos:
        dataframe = cargar_dataframe_ambulatorio(ruta_archivo)
        resultado_filtrado = filtro_dataframe_ca_tiroides(dataframe)
        resultado_filtrado = agregar_dv(resultado_filtrado)
        resultados = agregar_diagnosticos_y_codigos(resultado_filtrado)
        nuevo_dataframe = crear_dataframe_ambulatorio(resultado_filtrado)
        merged_df = merge_ambulatorio(resultados, nuevo_dataframe)
        nuevo_dataframe = filtrar_columnas_relevantes(merged_df)
        resultados_totales.append(nuevo_dataframe)
    return resultados_totales 

def proceso_dataframe_ultimo(ruta_archivo):
    dataframe = cargar_dataframe_ambulatorio(ruta_archivo)
    resultado_filtrado = filtro_dataframe_ca_tiroides(dataframe)
    resultado_filtrado = agregar_dv(resultado_filtrado)
    resultados = agregar_diagnosticos_y_codigos(resultado_filtrado)
    nuevo_dataframe = crear_dataframe_ambulatorio(resultado_filtrado)
    merged_df = merge_ambulatorio(resultados, nuevo_dataframe)
    nuevo_dataframe = filtrar_columnas_relevantes(merged_df)
    return nuevo_dataframe

def proceso_dataframe_ieeh(rutas_archivos):
    resultados_totales = []  # Lista para almacenar los resultados de todos los archivos
    for ruta_archivo in rutas_archivos:
        dataframe = cargar_dataframe_ieeh(ruta_archivo)
        resultado_filtrado = filtro_dataframe_ca_tiroides_ieeh(dataframe)
        resultados = agregar_diagnosticos_ieeh(resultado_filtrado)
        nuevo_dataframe = crear_dataframe_ieeh(resultado_filtrado)
        merged_df = merge_ambulatorio(resultados, nuevo_dataframe)
        nuevo_dataframe = filtrar_columnas_relevantes(merged_df)
        resultados_totales.append(nuevo_dataframe)
    return resultados_totales

def proceso_dataframe_urgencias(rutas_archivos):
    resultados_totales = []  # Lista para almacenar los resultados de todos los archivos
    for ruta_archivo in rutas_archivos:
        dataframe = cargar_dataframe_ieeh(ruta_archivo)
        resultado_filtrado = filtro_dataframe_ca_tiroides_urgencias(dataframe)
        resultado_filtrado = agregar_dv_urgencias(resultado_filtrado)
        resultados = agregar_diagnosticos_y_codigos_urgencias(resultado_filtrado)
        nuevo_dataframe = crear_dataframe_urgencias(resultado_filtrado)
        merged_df = merge_urgencia(resultados, nuevo_dataframe)
        nuevo_dataframe = filtrar_columnas_relevantes(merged_df)
        resultados_totales.append(nuevo_dataframe)
    return resultados_totales

def proceso_dataframe_inter(ruta_archivo):
    dataframe = cargar_dataframe_ieeh(ruta_archivo)
    dataframe['FECHA_SOLIC'] = pd.to_datetime(dataframe['FECHA_SOLIC'], format='%d/%m/%Y')
    dataframe = dataframe[dataframe['ESTADO_SOL'].isin(['Solicitada','Pendiente de Autorizar', 'Citada'])]
    dataframe = dataframe[~dataframe['NRO_IDENTIFICADOR'].str.contains('AT')]
    # Definir la fecha de inicio para el filtro
    fecha_hoy = datetime.today()
    fecha_inicio = fecha_hoy - timedelta(days=10)

    fecha_inicio = pd.to_datetime(fecha_inicio, format='%d/%m/%Y')
    # Filtrar el DataFrame
    dataframe = dataframe[dataframe['FECHA_SOLIC'] >= fecha_inicio]
    resultado_filtrado = filtro_dataframe_ca_tiroides_inter(dataframe)
    resultado_filtrado = agregar_dv_inter(resultado_filtrado)
    resultados = agregar_diagnosticos_y_codigos_inter(resultado_filtrado)
    nuevo_dataframe = crear_dataframe_inter(resultado_filtrado)
    merged_df = merge_urgencia(resultados, nuevo_dataframe)
    nuevo_dataframe = filtrar_columnas_relevantes(merged_df)
 
    return nuevo_dataframe

def ultimo_csv_en_carpeta(ruta):
    # Obtener lista de archivos en la carpeta
    archivos = os.listdir(ruta)
    
    # Filtrar archivos que sean .csv
    archivos_csv = [archivo for archivo in archivos if archivo.endswith('.csv')]
    
    if not archivos_csv:
        return None  # Si no hay archivos .csv en la carpeta
    
    # Ordenar la lista de archivos por fecha de modificación
    archivos_csv.sort(key=lambda x: os.path.getmtime(os.path.join(ruta, x)), reverse=True)
    
    # Devolver la ruta completa del último archivo .csv
    return os.path.join(ruta, archivos_csv[0])

def convertir_fecha(fecha):
    try:
        return pd.to_datetime(fecha, format='%d/%m/%Y %H:%M')
    except ValueError:
        return pd.to_datetime(fecha, format='%d-%m-%y %H:%M')

def obtener_fechas_amb(rutas_archivos):
    resultados_totales = []  # Lista para almacenar los resultados de todos los archivos
    for ruta_archivo in rutas_archivos:
        dataframe = cargar_dataframe_ambulatorio(ruta_archivo)
        dataframe['DV'] = dataframe['NRO_IDENTIFICADOR_PACIENTE'].str[-1]
        dataframe['RUT'] = dataframe['NRO_IDENTIFICADOR_PACIENTE'].str[:-1]
        



        resultados_totales.append(dataframe)
        
    dataframe = pd.concat(resultados_totales)

    
    # Filtrar por las especialidades de Endocrinología Adulto y Endocrinología Infantil
    endocrinologia_df = dataframe[dataframe['ESPECIALIDAD_AGENDA'].isin(['Endocrinología Adulto','Cirugia Cabeza y Cuello'])]
    endocrinologia_df = endocrinologia_df[endocrinologia_df['ESTAMENTO_PROFESIONAL_ATENCION']=='Médico']
    # Filtrar las filas que no contienen 'No realizada' o 'Anulada' en la columna 'ESTADO_CITA'
    endocrinologia_df = endocrinologia_df[~endocrinologia_df['ESTADO_CITA'].isin(['No realizada', 'Anulada'])]
    # Convertir la columna 'HORA_TERMINO_ATENCION' a formato de fecha y hora
    #endocrinologia_df['HORA_TERMINO_ATENCION'] = pd.to_datetime(endocrinologia_df['HORA_TERMINO_ATENCION'], format='mixed')
    endocrinologia_df['HORA_TERMINO_ATENCION'] = endocrinologia_df['HORA_TERMINO_ATENCION'].apply(convertir_fecha)

    # Agrupar por RUT y obtener la última fecha de la hora de atención para cada grupo
    #ultima_fecha_por_rut = endocrinologia_df.groupby('RUT').agg({'HORA_TERMINO_ATENCION': 'max'}).reset_index()

    ultima_fecha_por_rut =  endocrinologia_df.groupby('RUT').agg({
    'HORA_TERMINO_ATENCION': 'max',
    'NOMBRE_PROFESIONAL_ATENCION': 'first',
    'ESTAMENTO_PROFESIONAL_ATENCION': 'first',
    'DESCRIPCION': 'first',
    'ESPECIALIDAD_AGENDA': 'first'
}).reset_index()
    # Fusionar el DataFrame original con el DataFrame resultante para recuperar NOMBRE_PROFESIONAL_ATENCION y ESTAMENTO_PROFESIONAL_ATENCION
    resultado = ultima_fecha_por_rut.merge(endocrinologia_df[['RUT', 'HORA_TERMINO_ATENCION', 'NOMBRE_PROFESIONAL_ATENCION', 'ESTAMENTO_PROFESIONAL_ATENCION','DESCRIPCION']], on=['RUT', 'HORA_TERMINO_ATENCION'], how='left')

    return ultima_fecha_por_rut 

# Ruta archivos

In [4]:
dir_actual = os.getcwd()
dir_archivo_csv = os.path.join(dir_actual,'..','Reporteria.lnk')

In [5]:
import win32com.client

def get_target_path(lnk_file):
    shell = win32com.client.Dispatch("WScript.Shell")
    shortcut = shell.CreateShortCut(lnk_file)
    return shortcut.Targetpath

# Ruta del archivo de acceso directo (.lnk)
ruta_del_lnk = os.path.join(dir_actual,'..','Reporteria.lnk')
# Obtener la ruta de destino del archivo de acceso directo
ruta_reporteria = get_target_path(ruta_del_lnk)

#print("Ruta de destino:", ruta_reporteria)


In [6]:
# Ruta del archivo de acceso directo (.lnk)
ruta_del_lnk = os.path.join(dir_actual,'..','Unidad_Gestion_Del_Paciente.lnk')
# Obtener la ruta de destino del archivo de acceso directo
ruta_ges = get_target_path(ruta_del_lnk)

#print("Ruta de destino:", ruta_ges)

In [7]:
patron = os.path.join(ruta_reporteria,'Reportes_Ambulatorio','Diario', '*Reporte_Ambulatorio_Diario*')

archivos = glob.glob(patron)

archivos_ordenados = sorted(archivos, key=os.path.getmtime, reverse = True)

ultimos_10_archivos = archivos_ordenados[:10]

for archivo in ultimos_10_archivos:
    fecha_modificacion = datetime.fromtimestamp(os.path.getmtime(archivo))
    #print(f"{archivo}- modificación: {fecha_modificacion}")

In [8]:


#################### RUTA ARCHIVOS ##########################################

dias_reportes = 10

# Lista de rutas de archivos
rutas_archivos = []

# Día actual
today = date.today()

# Fecha actual
now = datetime.now()

# Formatear día con ceros a la izquierda si es necesario
day_formatted = "{:02d}".format(today.day)
# Formatear mes con ceros a la izquierda si es necesario
month_formatted = "{:02d}".format(today.month)


#Reportes Ambulatorios Histórico
ruta_archivo_4 = os.path.join(ruta_reporteria,'Reportes_Ambulatorio','Mensual', '202401_Reporte_Ambulatorio_Mensual.csv')
ruta_archivo_5 = os.path.join(ruta_reporteria,'Reportes_Ambulatorio','Diario', '20240401_Reporte_Ambulatorio_Diario.csv')


# Lista de rutas de archivos
rutas_archivos_hist = [ruta_archivo_4, ruta_archivo_5]
#agregamos los dias 
for i in range(1,int(day_formatted)+1):
        # Esto asegura que el número tenga dos dígitos, agregando un cero inicial si es necesario
    numero_formateado = f'{i:02}'
    #rutas_archivos.append(os.path.join(dir_datos_hlf, 'BDD_Abril', f'{today.year}{month_formatted}{numero_formateado}_Reporte_Ambulatorio_Diario.csv'))
    rutas_archivos_hist.append(os.path.join(ruta_reporteria,'Reportes_Ambulatorio','Diario', f'{today.year}{month_formatted}{numero_formateado}_Reporte_Ambulatorio_Diario.csv'))
for i in range(1,today.month):
        # Esto asegura que el número tenga dos dígitos, agregando un cero inicial si es necesario
    numero_formateado = f'{i:02}'
    #rutas_archivos.append(os.path.join(dir_datos_hlf, 'BDD_A_M', 'Mensual Ambulatorio', f'{today.year}{numero_formateado}_Reporte_Ambulatorio_Mensual.csv'))
    rutas_archivos_hist.append(os.path.join(ruta_reporteria,'Reportes_Ambulatorio','Mensual', f'{today.year}{numero_formateado}_Reporte_Ambulatorio_Mensual.csv'))

for i in range(1,6):
        # Esto asegura que el número tenga dos dígitos, agregando un cero inicial si es necesario
    numero_formateado = f'{i:02}'
    #rutas_archivos.append(os.path.join(dir_datos_hlf, 'BDD_A_M', 'Anual Ambulatorio', f'{today.year-i}_Reporte_Ambulatorio_Anual.csv'))
    rutas_archivos_hist.append(os.path.join(ruta_reporteria,'Reportes_Ambulatorio','Anual', f'{today.year-i}_Reporte_Ambulatorio_Anual.csv'))
  



#Reportes Ambulatorios


patron_amb = os.path.join(ruta_reporteria,'Reportes_Ambulatorio','Diario', '*Reporte_Ambulatorio_Diario*')

archivos_amb = glob.glob(patron_amb)

archivos_ordenados_amb = sorted(archivos_amb, key=os.path.getmtime, reverse = True)

rutas_archivos = archivos_ordenados_amb[:dias_reportes]



#Reportes IEEH

rutas_archivos_2 = []


patron_ieeh = os.path.join(ruta_reporteria,'Reporte_IEEH','Diario', '*Informe_Diario-IEEE*')

archivos_ieeh  = glob.glob(patron_ieeh )

archivos_ordenados_ieeh  = sorted(archivos_ieeh , key=os.path.getmtime, reverse = True)

rutas_archivos_2 = archivos_ordenados_ieeh[:dias_reportes]


#Reportes Urgencias


rutas_archivos_3 = []


patron_urg = os.path.join(ruta_reporteria,'Reportes_Produccion_Urgencia','Diario', '*Urgencias_atenciones*')

archivos_urg  = glob.glob(patron_urg)

archivos_ordenados_urg  = sorted(archivos_urg , key=os.path.getmtime, reverse = True)

rutas_archivos_3 = archivos_ordenados_urg[:dias_reportes]





In [9]:
#Reportes SIC

patron_sic = os.path.join(ruta_reporteria,'Reportes_Solicitudes','Mensual', '*Informe_Solicitudes_DerivacionCE*')

archivos_sic  = glob.glob(patron_sic)

archivos_ordenados_sic  = sorted(archivos_sic , key=os.path.getmtime, reverse = True)

rutas_archivos_4 = archivos_ordenados_sic[:1]


# Generar dataframes

In [10]:
# Llamar a la función proceso_dataframe con la lista de rutas de archivos
resultado = obtener_fechas_amb(rutas_archivos_hist)

resultado_final = proceso_dataframe(rutas_archivos)

resultado_final_2 = proceso_dataframe_ieeh(rutas_archivos_2)

resultado_final_3 = proceso_dataframe_urgencias(rutas_archivos_3)

resultado_final_4 = proceso_dataframe_inter(rutas_archivos_4[0])

In [11]:



#os.path.join(ruta_ges,'UNIDAD GES', 'SCRIPT_GES','Cruce GES','Cruce.xlsx')
ruta_del_archivo = os.path.join(ruta_ges,'UNIDAD GES', 'SCRIPT_GES','Cruce GES','Cruce.xlsx')


# Carga el archivo XLSX en un DataFrame
dataframe = pd.read_excel(ruta_del_archivo)

In [12]:
# Carga el archivo XLSX en un DataFrame
dataframe = pd.read_excel(ruta_del_archivo)

# Construir la ruta completa al archivo CSV
dataframe['RUT']= dataframe['RUT'].apply(lambda x : str(x))
dataframe['RUT COMPLETO'] = dataframe['RUT'].astype(str)  + dataframe['DV'].astype(str)
dataframe_tiroides = dataframe[dataframe['Problema de salud'] == 'Cáncer de Tiroides . {Decreto Nro 22/2019}']

dataframe_tiroides["Fecha inicio"]=pd.to_datetime(dataframe_tiroides["Fecha inicio"])
# Ordenar en orden descendente por la columna "FECHA_CITA"
dataframe_tiroides = dataframe_tiroides.sort_values(by='Fecha inicio', ascending=False)


# Eliminar duplicados basados en la columna "NRO_IDENTIFICADOR_PACIENTE"
dataframe_tiroides = dataframe_tiroides.drop_duplicates(subset='RUT', keep='first')

nuevo_dataframe_2 = crear_dataframe_ges(dataframe_tiroides)

resultado_final.append(nuevo_dataframe_2)



#if ultimo_csv:
#    print("La dirección del último archivo .csv en la carpeta es:", ultimo_csv)
    
#    resultado_final.append(proceso_dataframe_ultimo(ultimo_csv))
#    df_concatenated = pd.concat(resultado_final)

#else:
#    print("No se encontraron archivos .csv en la carpeta.")
#    df_concatenated = pd.concat(resultado_final)
 # Concatenar los DataFrames
resultado_final.extend(resultado_final_2)
resultado_final.extend(resultado_final_3)
resultado_final.extend([resultado_final_4])
df_concatenated = pd.concat(resultado_final)






# Eliminar duplicados basados en la columna "NRO_IDENTIFICADOR_PACIENTE"

#consolidated_df = df_concatenated.groupby('RUT').apply(lambda x: x.fillna(method='ffill').fillna(method='bfill')).drop_duplicates().reset_index(drop=True)

#resultado_filtrado_2 = consolidated_df.drop_duplicates(subset='RUT', keep='first')

resultado_filtrado_2 = df_concatenated.drop_duplicates(subset='RUT', keep='first')

# Iterar sobre las filas del primer DataFrame
for index, row in resultado_filtrado_2.iterrows():
    rut = row['RUT']
    
    # Buscar el valor correspondiente en el segundo DataFrame
    nombre_garantia = nuevo_dataframe_2.loc[nuevo_dataframe_2['RUT'] == str(rut), 'Nombre garantia'].values
    
    # Encontrar la fila correspondiente en el primer DataFrame y asignar el valor
    resultado_filtrado_2.loc[resultado_filtrado_2['RUT'] == rut, 'Nombre garantia'] = nombre_garantia[0] if nombre_garantia.size > 0 else None

resultado_filtrado_2['RUT'] = resultado_filtrado_2['RUT'].apply(lambda x: str(x))


In [13]:
#nombre_archivo = 'Total pacientes_nuevos_ca_tiroide.xlsx'  # Nombre del archivo de Excel
#resultado_filtrado_2.to_excel(nombre_archivo, index=False)  # index=False para no incluir el índice en el archivo de Excel


In [14]:
resultado_filtrado_2 = resultado_filtrado_2.drop_duplicates(subset='RUT', keep='first')

# Cruce BDD Gestoras

In [15]:
dir_actual = os.getcwd()
os.listdir(os.path.join(dir_actual, '..','CA TIROIDES'))

['20240510_con_ultima_atencion.xlsx',
 '20240701_citas_futuras_tiroides.xlsx',
 '20240701_ultima_atencion_tiroides.xlsx',
 'BASE TRABAJO GESTION ONCOLOGICA_CANCER TIROIDES.xlsx',
 'Reporte diario casos nuevos']

In [16]:
archivo_excel = os.path.join(dir_actual, '..','CA TIROIDES', 'BASE TRABAJO GESTION ONCOLOGICA_CANCER TIROIDES.xlsx') # Gestion_Oncologica -> apolo

dataframe_2 = pd.read_excel(archivo_excel, sheet_name='Data')


In [17]:
dataframe_2

,Problema Oncologico,Rut,DV,Nombre,Etapa,Estado,Fecha Próximo Hito,Medico Responsable,Fecha de Comité,Observacion,Diagnosticos,Telefonos,CITAS FUTURAS,MEDICO,Fecha Ultima atencion,Profesional Ultima atencion
0,Cancer de Tiroides,6053384,9,CLARA LUISA PINTO RAMÍREZ,Alta,Enviado a APS,2013-08-01 00:00:00,NaN,NaN,"DG HIPOTIROIDISMO, SEGÚN SIGGES EN HLF. SIN FI...",NaN,NaN,NaN,NaN,22-12-2023,Médico VERÓNICA CECILIA MARIANI MARTÍN Ecograf...
1,Cancer de Tiroides,8617827,3,NANCY DEL CARMEN MORALES DÍAZ,Alta,Enviado a APS,2013-08-19 00:00:00,NaN,NaN,PACIENTE CON DIAGNOSTICO DE HIPOTIROIDISMO ATE...,NaN,NaN,NaN,NaN,06-12-2023,Médico GERALDINE ELIZABETH MURATURE STORDIAU M...
2,Cancer de Tiroides,12116318,7,BERNARDITA JACQUELIN VALENZUELA FARÍAS,Alta,Enviado a APS,2013-09-24 00:00:00,NaN,NaN,PACIENTE CON SIGGES EN HIPOTIROIDISMO,NaN,NaN,NaN,NaN,NaN,NaN
3,Cancer de Tiroides,9480596,1,ANA ELIZABETH VENEGAS PORTILLA,Alta,Enviado a APS,2013-11-22 00:00:00,NaN,NaN,"HIPOTIROIDISMO EN GES DEL 2013, SIN IC",['M2 (PV)POPIOEANTEC CA TIROIDES OPERADO (2016...,959003560_/227912341/-,NaN,NaN,NaN,NaN
4,Cancer de Tiroides,5576350,K,CLEMENCIA RAQUEL LILLO FARJI,Alta,Enviado a APS,2014-04-23 00:00:00,NaN,NaN,DG DE HIPOTIROIDISMO EN SIGGES. SIN NADA EN TH...,['Hipotiroidismo consecutivo a procedimientos'...,-/-/-,NaN,NaN,11-09-2022,Médico DORA ALEXANDRA ABARCA RUIZ TC de Abdome...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
625,Cancer de Tiroides,9426037,K,ALEJANDRA SOLEDAD BRAVO SILVA,Seguimiento,Pendiente Revisar,NaN,revisar,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
626,Cancer de Tiroides,9487419,K,MANUEL ENRIQUE CONTRERAS BUSTAMANTE,Seguimiento,Pendiente Revisar,NaN,revisar,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
627,Cancer de Tiroides,16028379,3,ALEJANDRA LORENA FAÚNDEZ MARTÍNEZ,Seguimiento,Pendiente Revisar,NaN,revisar,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
628,Cancer de Tiroides,16976979,6,REGINA ANDREA PRADINE PARRAGUEZ,Seguimiento,Pendiente Revisar,NaN,revisar,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:

dataframe = resultado_filtrado_2.copy()

# Obtener la ruta del directorio actual del script
#dir_actual = os.path.dirname(__file__)
#dir_actual = os.getcwd()

dir_actual = os.getcwd()
os.listdir(os.path.join(dir_actual, '..','CA TIROIDES'))


# Construir la ruta completa al archivo CSV
archivo_excel = os.path.join(dir_actual, '..','CA TIROIDES', 'BASE TRABAJO GESTION ONCOLOGICA_CANCER TIROIDES.xlsx') # Gestion_Oncologica -> apolo

dataframe_2 = pd.read_excel(archivo_excel, sheet_name='Data')


# Obtener la cantidad de filas que no son nulas en la columna 'mi_columna'
cantidad_filas_no_nulas = dataframe_2['Rut'].count()



print("Cantidad de pacientes en la base", cantidad_filas_no_nulas)



dataframe_2 = dataframe_2[:cantidad_filas_no_nulas]

dataframe_2.reset_index(drop=True, inplace=True)


# Renombrar la columna 'old_name' a 'new_name'
dataframe_2 = dataframe_2.rename(columns={'Rut': 'RUT'})


dataframe_2 = dataframe_2.rename(columns={'Problema de Salud': 'Problema Oncologico'})

dataframe_2 = dataframe_2.rename(columns={'Nombre': 'Nombre del Paciente'})

# Definir una función para quitar los últimos dos dígitos
def quitar_ultimos_dos(elemento):
    return np.int64(elemento[:-2])

# Convertir todos los elementos de la columna 'RUT' a string
dataframe_2['RUT'] = dataframe_2['RUT'].astype(str)

# Aplicar la función a la columna 'RUT'
#dataframe_2['RUT'] = dataframe_2['RUT'].apply(quitar_ultimos_dos)



# Identificar las filas donde 'DV' es NaN
filas_dv_nan = dataframe_2[dataframe_2['DV'].isna()]

# Iterar sobre estas filas y corregir los valores de 'DV' y 'Rut'
for index, fila in filas_dv_nan.iterrows():
    rut = str(fila['RUT'])
    if True:
      dv = rut[-1]  # Último dígito del Rut
      dataframe_2.at[index, 'DV'] = dv
      dataframe_2.at[index, 'RUT'] = rut[:-1]  # Eliminar el último dígito del Rut
dataframe_2 = dataframe_2[~(dataframe_2['RUT'].str.contains('A', case=False, na=False))]

dataframe_2['RUT'] = dataframe_2['RUT'].astype(float)
dataframe_2['RUT'] = dataframe_2['RUT'].astype(int)


Cantidad de pacientes en la base 630


In [19]:
dataframe_2

,Problema Oncologico,RUT,DV,Nombre del Paciente,Etapa,Estado,Fecha Próximo Hito,Medico Responsable,Fecha de Comité,Observacion,Diagnosticos,Telefonos,CITAS FUTURAS,MEDICO,Fecha Ultima atencion,Profesional Ultima atencion
0,Cancer de Tiroides,6053384,9,CLARA LUISA PINTO RAMÍREZ,Alta,Enviado a APS,2013-08-01 00:00:00,NaN,NaN,"DG HIPOTIROIDISMO, SEGÚN SIGGES EN HLF. SIN FI...",NaN,NaN,NaN,NaN,22-12-2023,Médico VERÓNICA CECILIA MARIANI MARTÍN Ecograf...
1,Cancer de Tiroides,8617827,3,NANCY DEL CARMEN MORALES DÍAZ,Alta,Enviado a APS,2013-08-19 00:00:00,NaN,NaN,PACIENTE CON DIAGNOSTICO DE HIPOTIROIDISMO ATE...,NaN,NaN,NaN,NaN,06-12-2023,Médico GERALDINE ELIZABETH MURATURE STORDIAU M...
2,Cancer de Tiroides,12116318,7,BERNARDITA JACQUELIN VALENZUELA FARÍAS,Alta,Enviado a APS,2013-09-24 00:00:00,NaN,NaN,PACIENTE CON SIGGES EN HIPOTIROIDISMO,NaN,NaN,NaN,NaN,NaN,NaN
3,Cancer de Tiroides,9480596,1,ANA ELIZABETH VENEGAS PORTILLA,Alta,Enviado a APS,2013-11-22 00:00:00,NaN,NaN,"HIPOTIROIDISMO EN GES DEL 2013, SIN IC",['M2 (PV)POPIOEANTEC CA TIROIDES OPERADO (2016...,959003560_/227912341/-,NaN,NaN,NaN,NaN
4,Cancer de Tiroides,5576350,K,CLEMENCIA RAQUEL LILLO FARJI,Alta,Enviado a APS,2014-04-23 00:00:00,NaN,NaN,DG DE HIPOTIROIDISMO EN SIGGES. SIN NADA EN TH...,['Hipotiroidismo consecutivo a procedimientos'...,-/-/-,NaN,NaN,11-09-2022,Médico DORA ALEXANDRA ABARCA RUIZ TC de Abdome...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
625,Cancer de Tiroides,9426037,K,ALEJANDRA SOLEDAD BRAVO SILVA,Seguimiento,Pendiente Revisar,NaN,revisar,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
626,Cancer de Tiroides,9487419,K,MANUEL ENRIQUE CONTRERAS BUSTAMANTE,Seguimiento,Pendiente Revisar,NaN,revisar,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
627,Cancer de Tiroides,16028379,3,ALEJANDRA LORENA FAÚNDEZ MARTÍNEZ,Seguimiento,Pendiente Revisar,NaN,revisar,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
628,Cancer de Tiroides,16976979,6,REGINA ANDREA PRADINE PARRAGUEZ,Seguimiento,Pendiente Revisar,NaN,revisar,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Merge Total pacientes vs Gestoras

In [20]:
# Ejemplo de creación de DataFrame (puedes cargar tus propios DataFrames)
df1 = dataframe.copy()
df2 = dataframe_2.copy()

# Agrega una columna para identificar el origen de cada RUT
df1['_df'] = 'df1'
df2['_df'] = 'df2'

# Convertir la columna 'RUT' en ambos DataFrames al mismo tipo de datos
df1['RUT'] = df1['RUT'].astype(str)
df2['RUT'] = df2['RUT'].astype(str)

#print(f"df_1: {df1['RUT'] }")
#print(f"df_2: {df2['RUT'] }")

# Realiza un merge de los dos DataFrames en la columna 'RUT' y conserva solo las filas que aparecen en 'left_only'
df_merge = pd.merge(df1, df2, on='RUT', how='outer', indicator=True)


# Combinar las columnas 'DV_x' y 'DV_y' en una sola columna 'DV'
df_merge['Nombre del Paciente_x'].fillna(df_merge['Nombre del Paciente_y'], inplace=True)

# Renombrar la columna 'DV_x' a 'DV'
df_merge.rename(columns={'Nombre del Paciente_x': 'Nombre del Paciente'}, inplace=True)

# Eliminar la columna 'DV_y' si ya no es necesaria
df_merge.drop(columns=['Nombre del Paciente_y'], inplace=True)

# Combinar las columnas 'DV_x' y 'DV_y' en una sola columna 'DV'
df_merge['Problema Oncologico_x'].fillna(df_merge['Problema Oncologico_y'], inplace=True)

# Renombrar la columna 'DV_x' a 'DV'
df_merge.rename(columns={'Problema Oncologico_x': 'Problema Oncologico'}, inplace=True)

# Eliminar la columna 'DV_y' si ya no es necesaria
df_merge.drop(columns=['Problema Oncologico_y'], inplace=True)


# Combinar las columnas 'DV_x' y 'DV_y' en una sola columna 'DV'
df_merge['DV_x'].fillna(df_merge['DV_y'], inplace=True)

# Renombrar la columna 'DV_x' a 'DV'
df_merge.rename(columns={'DV_x': 'DV'}, inplace=True)

# Eliminar la columna 'DV_y' si ya no es necesaria
df_merge.drop(columns=['DV_y'], inplace=True)

df_ca_tiroide = df_merge[df_merge['_merge']=='left_only']

# Reemplazar los valores NaN por '-'
df_ca_tiroide['N° Telefono 1'] = df_ca_tiroide['N° Telefono 1'].fillna('-')
df_ca_tiroide['N° Telefono 2'] = df_ca_tiroide['N° Telefono 2'].fillna('-')
df_ca_tiroide['N° Telefono 3'] = df_ca_tiroide['N° Telefono 3'].fillna('-')

def combinar_telefonos(row):
    return '/'.join(filter(None, [str(row['N° Telefono 1']), str(row['N° Telefono 2']), str(row['N° Telefono 3'])]))

# Aplicar la función a lo largo de las filas del DataFrame para crear la columna 'Telefono Consolidado'
df_ca_tiroide_telefono = df_ca_tiroide[['N° Telefono 1','N° Telefono 2','N° Telefono 3']]
df_ca_tiroide['Telefono Consolidado2'] = df_ca_tiroide_telefono.apply(combinar_telefonos, axis=1)

# Eliminar las columnas originales de los números de teléfono si ya no son necesarias
df_ca_tiroide.drop(columns=['N° Telefono 1', 'N° Telefono 2', 'N° Telefono 3'], inplace=True)

df_ca_tiroide['Etapa'] = np.nan

df_ca_tiroide['Estado'] = np.nan

df_ca_tiroide['Fecha Proximo Hito'] = np.nan

df_ca_tiroide['Observacion'] = np.nan

df_ca_tiroide['Medico Responsable'] = np.nan

df_ca_tiroide['Fecha Ultima atencion'] = np.nan

df_ca_tiroide['Profesional Ultima atencion'] = np.nan

df_ca_tiroide['MEDICO'] = np.nan

df_ca_tiroide['CITAS FUTURAS'] = np.nan

#df_ca_tiroide = df_ca_tiroide[['Problema Oncologico', 'RUT', 'DV', 'Nombre del Paciente','Etapa','Nombre garantia','Estado','Fecha Proximo Hito','Observacion','Medico Responsable','Fecha Ultima atencion', 'Profesional Ultima atencion','DIAGNOSTICOS', 'Telefono Consolidado2']]
df_ca_tiroide = df_ca_tiroide[['Problema Oncologico',
 'RUT',
 'DV',
 'Nombre del Paciente',
 'Etapa',
 'Estado',
 'Fecha Próximo Hito',
 'Observacion',
 'Medico Responsable',
 'DIAGNOSTICOS',
 'Telefono Consolidado2',
 'CITAS FUTURAS',
 'MEDICO',
 'Fecha Ultima atencion',
 'Profesional Ultima atencion']]

In [21]:
df_ca_tiroide

,Problema Oncologico,RUT,DV,Nombre del Paciente,Etapa,Estado,Fecha Próximo Hito,Observacion,Medico Responsable,DIAGNOSTICOS,Telefono Consolidado2,CITAS FUTURAS,MEDICO,Fecha Ultima atencion,Profesional Ultima atencion
494,Cancer de Tiroides,6924130,1,ELSA VICTORIA ÁLVAREZ ORELLANA,NaN,NaN,NaN,NaN,NaN,[Tumor de comportamiento incierto o desconocid...,76491061/76491061/-,NaN,NaN,NaN,NaN
580,Cancer de Tiroides,9035912,6,ERIKA VIVIANA ROJAS VALENZUELA,NaN,NaN,NaN,NaN,NaN,[M2 (2CCA)DCCHHTA CRÓNICA EN TTOREFLUJO GASTRO...,92398618/29816009/-,NaN,NaN,NaN,NaN
588,Cancer de Tiroides,9305762,7,ELEANOR RUTH ZÚÑIGA VICUÑA,NaN,NaN,NaN,NaN,NaN,[Tumor de comportamiento incierto o desconocid...,0996256 3413/022454 3836/-,NaN,NaN,NaN,NaN
621,Cancer de Tiroides,9766888,4,PILAR ANTONIETA RETAMALES HERNÁNDEZ,NaN,NaN,NaN,NaN,NaN,"[Tumor maligno de la glandula paratiroides, Hi...",45995688.0/88796605/-,NaN,NaN,NaN,NaN


In [22]:
df_ca_tiroide['Fecha de Comité'] = np.nan

In [23]:
lista = ['Problema Oncologico', 'RUT','DV','Nombre del Paciente','Etapa','Estado','Fecha Próximo Hito','Medico Responsable','Fecha de Comité','Observacion','DIAGNOSTICOS','Telefono Consolidado2','CITAS FUTURAS','MEDICO','Fecha Ultima atencion','Profesional Ultima atencion']

In [24]:
print(f'Cantidad de pacientes nuevos encontrados: {df_ca_tiroide.shape[0]}')

Cantidad de pacientes nuevos encontrados: 4


# Guardar estadística de llegada


In [49]:
fecha_hoy = date.today()
data = {'Problema Oncologico': ['Cancer de Tiroides'], 'Fecha Medición': [fecha_hoy], 'Cantidad de llegada': [df_ca_tiroide.shape[0]], 'Cantidad de pacientes base': [cantidad_filas_no_nulas]}
estadisticas_llegadas = pd.DataFrame(data)

nombre_archivo = f'Estadisticas_llegadas.xlsx'
archivo_excel = os.path.join(dir_actual, '..','Power BI',f'{nombre_archivo}')

df_existente = pd.read_excel(archivo_excel)
df_existente['Fecha Medición'] = pd.to_datetime(df_existente['Fecha Medición'])


wb = Workbook()
ws = wb.active
df_final = pd.concat([df_existente,estadisticas_llegadas], ignore_index=True)


for r in dataframe_to_rows(df_final, index= False, header = True):
    ws.append(r)

for cell in ws['B'][1:]:
    cell.number_format = 'DD-MM-YYYY'
nombre_archivo = f'Estadisticas_llegadas.xlsx'
wb.save(os.path.join(dir_actual, '..','Power BI',f'{nombre_archivo}'))


# Agregar Fallecidos

In [25]:
dir_actual = os.getcwd()
dir_gestoras = os.path.join(dir_actual,'..')

In [26]:
ruta_del_archivo_2 = os.path.join(dir_gestoras, 'FALLECIDOS HLF CORTE 30_04_2024.xlsx')


In [27]:
# Carga el archivo XLSX en un DataFrame
fallecidos = pd.read_excel(ruta_del_archivo_2)

In [28]:
fallecidos_rut = pd.DataFrame({'RUT': fallecidos['RUD']})

In [29]:
import pandas as pd

# Suponiendo que tus dataframes se llaman df_consolidado y df_fallecidos


# Combinamos los dataframes en función de la columna 'RUT'


fallecidos_rut['RUT'] = fallecidos_rut['RUT'].astype(str)  # Convertir la columna a tipo string
fallecidos_rut['RUT'] = fallecidos_rut['RUT'].replace(['-', '\.'], '', regex=True) 

fallecidos_rut['RUT'] = fallecidos_rut['RUT'].astype(str)


In [30]:
df_pacientes_nuevos = df_ca_tiroide[lista].copy()

In [31]:
df_pacientes_nuevos

,Problema Oncologico,RUT,DV,Nombre del Paciente,Etapa,Estado,Fecha Próximo Hito,Medico Responsable,Fecha de Comité,Observacion,DIAGNOSTICOS,Telefono Consolidado2,CITAS FUTURAS,MEDICO,Fecha Ultima atencion,Profesional Ultima atencion
494,Cancer de Tiroides,6924130,1,ELSA VICTORIA ÁLVAREZ ORELLANA,NaN,NaN,NaN,NaN,NaN,NaN,[Tumor de comportamiento incierto o desconocid...,76491061/76491061/-,NaN,NaN,NaN,NaN
580,Cancer de Tiroides,9035912,6,ERIKA VIVIANA ROJAS VALENZUELA,NaN,NaN,NaN,NaN,NaN,NaN,[M2 (2CCA)DCCHHTA CRÓNICA EN TTOREFLUJO GASTRO...,92398618/29816009/-,NaN,NaN,NaN,NaN
588,Cancer de Tiroides,9305762,7,ELEANOR RUTH ZÚÑIGA VICUÑA,NaN,NaN,NaN,NaN,NaN,NaN,[Tumor de comportamiento incierto o desconocid...,0996256 3413/022454 3836/-,NaN,NaN,NaN,NaN
621,Cancer de Tiroides,9766888,4,PILAR ANTONIETA RETAMALES HERNÁNDEZ,NaN,NaN,NaN,NaN,NaN,NaN,"[Tumor maligno de la glandula paratiroides, Hi...",45995688.0/88796605/-,NaN,NaN,NaN,NaN


In [32]:
df_pacientes_nuevos.loc[df_pacientes_nuevos['RUT'].isin(fallecidos_rut['RUT']), 'Estado'] = 'Fallecido'


In [33]:
df_pacientes_nuevos.loc[df_pacientes_nuevos['RUT'].isin(fallecidos_rut['RUT']), 'Etapa'] = 'Alta'


In [34]:
df_pacientes_nuevos = df_pacientes_nuevos[~(df_pacientes_nuevos['RUT']=='nan')]

In [35]:
df_pacientes_nuevos

,Problema Oncologico,RUT,DV,Nombre del Paciente,Etapa,Estado,Fecha Próximo Hito,Medico Responsable,Fecha de Comité,Observacion,DIAGNOSTICOS,Telefono Consolidado2,CITAS FUTURAS,MEDICO,Fecha Ultima atencion,Profesional Ultima atencion
494,Cancer de Tiroides,6924130,1,ELSA VICTORIA ÁLVAREZ ORELLANA,NaN,NaN,NaN,NaN,NaN,NaN,[Tumor de comportamiento incierto o desconocid...,76491061/76491061/-,NaN,NaN,NaN,NaN
580,Cancer de Tiroides,9035912,6,ERIKA VIVIANA ROJAS VALENZUELA,NaN,NaN,NaN,NaN,NaN,NaN,[M2 (2CCA)DCCHHTA CRÓNICA EN TTOREFLUJO GASTRO...,92398618/29816009/-,NaN,NaN,NaN,NaN
588,Cancer de Tiroides,9305762,7,ELEANOR RUTH ZÚÑIGA VICUÑA,NaN,NaN,NaN,NaN,NaN,NaN,[Tumor de comportamiento incierto o desconocid...,0996256 3413/022454 3836/-,NaN,NaN,NaN,NaN
621,Cancer de Tiroides,9766888,4,PILAR ANTONIETA RETAMALES HERNÁNDEZ,NaN,NaN,NaN,NaN,NaN,NaN,"[Tumor maligno de la glandula paratiroides, Hi...",45995688.0/88796605/-,NaN,NaN,NaN,NaN


# Guardar dataframe

In [25]:
dir_actual = os.getcwd()
os.listdir(os.path.join(dir_actual, '..','CA TIROIDES','Reporte diario casos nuevos'))

['20240624_Pacientes_nuevos_ca_tiroides.xlsx']

In [38]:
# Guardar el DataFrame en un archivo de Excel

# Día actual
today = date.today()

# Fecha actual
now = datetime.now()

# Formatear día con ceros a la izquierda si es necesario
day_formatted = "{:02d}".format(today.day)

# Formatear mes con ceros a la izquierda si es necesario
month_formatted = "{:02d}".format(today.month)


nombre_archivo = f'{today.year}{month_formatted}{day_formatted}_Pacientes_nuevos_ca_tiroides.xlsx'  # Nombre del archivo de Excel
df_pacientes_nuevos.to_excel(os.path.join(dir_actual, '..','CA TIROIDES','Reporte diario casos nuevos',f'{nombre_archivo}'), index=False)  # index=False para no incluir el índice en el archivo de Excel 

# Convertir la columna 'RUT' en DataFrame_2 a tipo object si es necesario


df_concatenated_2 = pd.concat([dataframe_2,df_pacientes_nuevos])

#print(list[df_concatenated_2.columns])




df_concatenated_2['RUT'] = df_concatenated_2['RUT'].astype(str)
resultado['RUT'] = resultado['RUT'].astype(str)
# Fusionar DataFrame_2 con el DataFrame de resultado utilizando la columna 'RUT' como clave de fusión

dataframe_final = df_concatenated_2.merge(resultado, on='RUT', how='left')

#print(list[dataframe_final.columns])


dataframe_final_2 = crear_dataframe_ultima_atencion(dataframe_final)

In [39]:
df_concatenated_2

,Problema Oncologico,RUT,DV,Nombre del Paciente,Etapa,Estado,Fecha Próximo Hito,Medico Responsable,Fecha de Comité,Observacion,...,Telefonos,CITAS FUTURAS,MEDICO,Fecha Ultima atencion,Profesional Ultima atencion,Columna1,Columna2,Columna3,DIAGNOSTICOS,Telefono Consolidado2
0,Cancer de Tiroides,18390687,9,"SOTO CÁRDENAS, JAVIERA FERNANDA",Seguimiento,Pendiente Control,2021-06-15 00:00:00,DRA GOENS,NaN,"paciente se atiende en uc, SIN CONTROL DESDE 2021",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Cancer de Tiroides,9422872,7,JUAN CARLOS ESPINOZA ZÁRATE,Alta,Enviado a APS,2022-08-02 00:00:00,DRA GARCIA,NaN,"PACIENTE CON CONTROL DE HIPOTIROIDISMO, OPERAD...",...,49091010/29840041/-,NaN,NaN,08-02-2022,Médico NATALIA MARGARITA GARCÍA SALAZAR CONSUL...,NaN,NaN,NaN,NaN,NaN
2,Cancer de Tiroides,15970807,1,"RIQUELME MEZA, CECILIA NOEMÍ",Seguimiento,Pendiente Control,2022-08-01 00:00:00,DRA GARCIA,NaN,DRA EVALUA Y DECIDE CONTROL EN 6 MESES. DESDE ...,...,NaN,NaN,NaN,22-03-2022,Médico NATALIA MARGARITA GARCÍA SALAZAR Ecogra...,NaN,NaN,NaN,NaN,NaN
3,Cancer de Tiroides,26885215,8,"SOSA , OMAIRA COROMOTO",Realizar tratamiento,Pendiente Control,2022-03-30 00:00:00,DR ARBULO,NaN,ULTIMA ATENION ENDOCRINA DRA GOENS DERIVA A CY...,...,NaN,NaN,NaN,30-03-2022,Médico MARÍA CRISTINA GOENS GUZMÁN Ecografia T...,NaN,NaN,NaN,NaN,NaN
4,Cancer de Tiroides,8128367,2,"VILLALOBOS MANZANO, ISABEL SOLEDAD",Alta,Enviado a APS,2021-08-11 00:00:00,NaN,NaN,SIN ATENCION DESDE 2021,...,NaN,NaN,NaN,01-04-2022,Médico DOMINGO FERNANDO SAAVEDRA PINTO Rx Col....,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
510,Cancer de Tiroides,13468630,8,ANDREA LILIANA MERLO BRAVO,ALTA,Enviado a APS,2023-07-05 00:00:00,DRA GOENS,NaN,DRA EVALUA Y DECIDE ALTA A APS.,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
511,Cancer de Tiroides,16611489,6,PAULINA ALEJANDRA ALEGRÍA VENEGAS,ingreso,Pendiente Ingreso,llamar,NaN,NaN,sin ic sin dg de ca tiroides,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
512,Cancer de Tiroides,6382061,k,CAROLINA MARGARITA RODRÍGUEZ ESPINOZA,ALTA,Alta oncologica,15-05-2024,DR SALAS,NaN,DG DE HIPOTIROIDISMO EN THALAMUS.,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20,Cancer de Tiroides,10520930,4,SANDRA DEL TRÁNSITO GARRIDO RODRÍGUEZ,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[Tumor maligno del colon, parte no especificad...",0984469994/981863591/-


In [40]:
dataframe_final_2

,RUT,Fecha Ultima atencion,Profesional Ultima atencion
0,18390687,NaT,NaN
1,9422872,2022-08-02 15:31:00,Médico NATALIA MARGARITA GARCÍA SALAZAR CONSUL...
2,15970807,2022-08-01 10:59:00,Médico NATALIA MARGARITA GARCÍA SALAZAR Ecogra...
3,26885215,2022-03-30 10:18:00,Médico MARÍA CRISTINA GOENS GUZMÁN Ecografia T...
4,8128367,2022-01-04 12:22:00,Médico DOMINGO FERNANDO SAAVEDRA PINTO Rx Col....
...,...,...,...
510,13468630,NaT,NaN
511,16611489,NaT,NaN
512,6382061,2024-05-15 14:13:00,Médico ANDRÉS IGNACIO SALAS FUENZALIDA Control...
513,10520930,NaT,NaN


In [41]:
# Convertir la columna de fecha a formato datetime
dataframe_final_2['Fecha Ultima atencion'] = pd.to_datetime(dataframe_final_2['Fecha Ultima atencion'])

# Formatear la columna de fecha en el formato deseado
dataframe_final_2['Fecha Ultima atencion'] = dataframe_final_2['Fecha Ultima atencion'].dt.strftime('%Y-%m-%d')


In [42]:
dataframe_final_2 = dataframe_final_2.dropna()

In [43]:
dataframe_final_2

,RUT,Fecha Ultima atencion,Profesional Ultima atencion
1,9422872,2022-08-02,Médico NATALIA MARGARITA GARCÍA SALAZAR CONSUL...
2,15970807,2022-08-01,Médico NATALIA MARGARITA GARCÍA SALAZAR Ecogra...
3,26885215,2022-03-30,Médico MARÍA CRISTINA GOENS GUZMÁN Ecografia T...
4,8128367,2022-01-04,Médico DOMINGO FERNANDO SAAVEDRA PINTO Rx Col....
5,5078415,2023-01-04,Médico MARÍA CRISTINA GOENS GUZMÁN Eco partes ...
...,...,...,...
428,6368256,2023-10-30,Médico ANIBAL CARBALLOSA RODRIGUEZ Rx Tórax PA...
429,12099710,2023-11-30,Médico MARÍA CRISTINA GOENS GUZMÁN Control - C...
439,20455288,2024-05-16,Médico Zeyuan Zhang TC de Abdomen y Pelvis co...
440,7979824,2024-05-25,Médico MARÍA CRISTINA GOENS GUZMÁN Control - C...


In [44]:
# Convertir la columna de fecha a formato datetime
dataframe_final_2['Fecha Ultima atencion'] = pd.to_datetime(dataframe_final_2['Fecha Ultima atencion'])

# Formatear la columna de fecha en el formato deseado
dataframe_final_2['Fecha Ultima atencion'] = dataframe_final_2['Fecha Ultima atencion'].dt.strftime('%d-%m-%Y')


In [45]:
dataframe_final_2

,RUT,Fecha Ultima atencion,Profesional Ultima atencion
1,9422872,02-08-2022,Médico NATALIA MARGARITA GARCÍA SALAZAR CONSUL...
2,15970807,01-08-2022,Médico NATALIA MARGARITA GARCÍA SALAZAR Ecogra...
3,26885215,30-03-2022,Médico MARÍA CRISTINA GOENS GUZMÁN Ecografia T...
4,8128367,04-01-2022,Médico DOMINGO FERNANDO SAAVEDRA PINTO Rx Col....
5,5078415,04-01-2023,Médico MARÍA CRISTINA GOENS GUZMÁN Eco partes ...
...,...,...,...
428,6368256,30-10-2023,Médico ANIBAL CARBALLOSA RODRIGUEZ Rx Tórax PA...
429,12099710,30-11-2023,Médico MARÍA CRISTINA GOENS GUZMÁN Control - C...
439,20455288,16-05-2024,Médico Zeyuan Zhang TC de Abdomen y Pelvis co...
440,7979824,25-05-2024,Médico MARÍA CRISTINA GOENS GUZMÁN Control - C...


In [46]:
import pandas
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows

In [47]:
wb = Workbook()
ws = wb.active

for r in dataframe_to_rows(dataframe_final_2, index= False, header = True):
    ws.append(r)

for cell in ws['B'][1:]:
    cell.number_format = 'DD-MM-YYYY'
nombre_archivo = f'{today.year}{month_formatted}{day_formatted}_ultima_atencion_tiroides.xlsx'
wb.save(os.path.join(dir_actual, '..','CA TIROIDES',f'{nombre_archivo}'))

# Guardar dataframe ultima atención 

In [48]:
#nombre_archivo = f'{today.year}{month_formatted}{day_formatted}_ultima_atencion_tiroides.xlsx'  # Nombre del archivo de Excel
#dataframe_final_2.to_excel(os.path.join(dir_actual, '..','CA TIROIDES',f'{nombre_archivo}'), index=False)  # index=False para no incluir el índice en el archivo de Excel 


In [49]:
df1 = df_concatenated_2.copy()
df2 = dataframe_final_2.copy()

df1['RUT'] = df1['RUT'].astype(str)
df2['RUT'] = df2['RUT'].astype(str)


# Realizar la fusión utilizando la columna 'RUT'
df_merged = pd.merge(df1, df2, on='RUT', how='left')



In [50]:
df2

,RUT,Fecha Ultima atencion,Profesional Ultima atencion
1,9422872,02-08-2022,Médico NATALIA MARGARITA GARCÍA SALAZAR CONSUL...
2,15970807,01-08-2022,Médico NATALIA MARGARITA GARCÍA SALAZAR Ecogra...
3,26885215,30-03-2022,Médico MARÍA CRISTINA GOENS GUZMÁN Ecografia T...
4,8128367,04-01-2022,Médico DOMINGO FERNANDO SAAVEDRA PINTO Rx Col....
5,5078415,04-01-2023,Médico MARÍA CRISTINA GOENS GUZMÁN Eco partes ...
...,...,...,...
428,6368256,30-10-2023,Médico ANIBAL CARBALLOSA RODRIGUEZ Rx Tórax PA...
429,12099710,30-11-2023,Médico MARÍA CRISTINA GOENS GUZMÁN Control - C...
439,20455288,16-05-2024,Médico Zeyuan Zhang TC de Abdomen y Pelvis co...
440,7979824,25-05-2024,Médico MARÍA CRISTINA GOENS GUZMÁN Control - C...


In [51]:
len(df1['RUT'].unique())

515

In [52]:
len(df2['RUT'].unique())

432

In [53]:
# Renombrar columnas para evitar colisiones
df_merged.rename(columns={'Fecha Ultima atencion_y': 'Fecha Ultima atencion', 'Profesional Ultima atencion_y': 'Profesional Ultima atencion'}, inplace=True)

# Eliminar columnas duplicadas
df_merged.drop(['Fecha Ultima atencion_x', 'Profesional Ultima atencion_x'], axis=1, inplace=True)



In [54]:
df_merged[df_merged.duplicated(subset=['RUT'], keep=False)]

,Problema Oncologico,RUT,DV,Nombre del Paciente,Etapa,Estado,Fecha Próximo Hito,Medico Responsable,Fecha de Comité,Observacion,...,Telefonos,CITAS FUTURAS,MEDICO,Columna1,Columna2,Columna3,DIAGNOSTICOS,Telefono Consolidado2,Fecha Ultima atencion,Profesional Ultima atencion


In [55]:
df_merged = df_merged.drop_duplicates(subset=['RUT'], keep='first')


In [56]:
df_merged

,Problema Oncologico,RUT,DV,Nombre del Paciente,Etapa,Estado,Fecha Próximo Hito,Medico Responsable,Fecha de Comité,Observacion,...,Telefonos,CITAS FUTURAS,MEDICO,Columna1,Columna2,Columna3,DIAGNOSTICOS,Telefono Consolidado2,Fecha Ultima atencion,Profesional Ultima atencion
0,Cancer de Tiroides,18390687,9,"SOTO CÁRDENAS, JAVIERA FERNANDA",Seguimiento,Pendiente Control,2021-06-15 00:00:00,DRA GOENS,NaN,"paciente se atiende en uc, SIN CONTROL DESDE 2021",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Cancer de Tiroides,9422872,7,JUAN CARLOS ESPINOZA ZÁRATE,Alta,Enviado a APS,2022-08-02 00:00:00,DRA GARCIA,NaN,"PACIENTE CON CONTROL DE HIPOTIROIDISMO, OPERAD...",...,49091010/29840041/-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,02-08-2022,Médico NATALIA MARGARITA GARCÍA SALAZAR CONSUL...
2,Cancer de Tiroides,15970807,1,"RIQUELME MEZA, CECILIA NOEMÍ",Seguimiento,Pendiente Control,2022-08-01 00:00:00,DRA GARCIA,NaN,DRA EVALUA Y DECIDE CONTROL EN 6 MESES. DESDE ...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,01-08-2022,Médico NATALIA MARGARITA GARCÍA SALAZAR Ecogra...
3,Cancer de Tiroides,26885215,8,"SOSA , OMAIRA COROMOTO",Realizar tratamiento,Pendiente Control,2022-03-30 00:00:00,DR ARBULO,NaN,ULTIMA ATENION ENDOCRINA DRA GOENS DERIVA A CY...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30-03-2022,Médico MARÍA CRISTINA GOENS GUZMÁN Ecografia T...
4,Cancer de Tiroides,8128367,2,"VILLALOBOS MANZANO, ISABEL SOLEDAD",Alta,Enviado a APS,2021-08-11 00:00:00,NaN,NaN,SIN ATENCION DESDE 2021,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,04-01-2022,Médico DOMINGO FERNANDO SAAVEDRA PINTO Rx Col....
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
510,Cancer de Tiroides,13468630,8,ANDREA LILIANA MERLO BRAVO,ALTA,Enviado a APS,2023-07-05 00:00:00,DRA GOENS,NaN,DRA EVALUA Y DECIDE ALTA A APS.,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
511,Cancer de Tiroides,16611489,6,PAULINA ALEJANDRA ALEGRÍA VENEGAS,ingreso,Pendiente Ingreso,llamar,NaN,NaN,sin ic sin dg de ca tiroides,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
512,Cancer de Tiroides,6382061,k,CAROLINA MARGARITA RODRÍGUEZ ESPINOZA,ALTA,Alta oncologica,15-05-2024,DR SALAS,NaN,DG DE HIPOTIROIDISMO EN THALAMUS.,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15-05-2024,Médico ANDRÉS IGNACIO SALAS FUENZALIDA Control...
513,Cancer de Tiroides,10520930,4,SANDRA DEL TRÁNSITO GARRIDO RODRÍGUEZ,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,"[Tumor maligno del colon, parte no especificad...",0984469994/981863591/-,NaN,NaN


# Crear Citas futuras

In [57]:

# Día actual
today = date.today()

# Fecha actual
now = datetime.now()

# Formatear día con ceros a la izquierda si es necesario
day_formatted = "{:02d}".format(today.day)

# Formatear mes con ceros a la izquierda si es necesario
month_formatted = "{:02d}".format(today.month)



ruta_del_archivo = os.path.join(ruta_reporteria,'Reportes_Ambulatorio','AgendasFuturas', f'{today.year}{month_formatted}{day_formatted}_Citas_Futuras.csv')


# Carga el archivo XLSX en un DataFrame
dataframe_fu = cargar_dataframe_ambulatorio(ruta_del_archivo)

In [58]:
# Mover los nombres de las columnas una posición hacia la izquierda
new_columns = list(dataframe_fu.columns)
new_columns = new_columns[1:] + [new_columns[0]]
dataframe_fu.columns = new_columns


In [59]:

dataframe_fu

,NOMBRE_AGENDA,ESPECIALIDAD_AGENDA,CITAFECH,CITAHORA,ESTADO_CITA,DESCRIPCION,ORIGEN,GES_CITA,GES_SOL,PATOLOGIA_GES_SOL,...,MOVIL_HABITUAL,TELEFONO1_HABITUAL,TELEFONO2_HABITUAL,TELEFONO3_HABITUALDIRECCION_HABITUAL,COMUNA_HABITUAL,MOVIL_CONTACTO,TELEFONO1_CONTACTO,TELEFONO2_CONTACTO,TELEFONO3_CONTACTO,CITA_KEY
1713458226593310985,DR. MAXIMILIANOIGNACIO LÓPEZ BETANZO,Otorrinolaringología,29-05-2024,1520,Agendada,Control - Consulta integral de especialidades,CEX,N,N,NaN,...,78023791,950157989,NaN,NaN,los nogales,La Florida,78023791,950157989,NaN,NaN
1713458226593310985,DR. MAXIMILIANOIGNACIO LÓPEZ BETANZO,Otorrinolaringología,29-05-2024,1520,Agendada,Control - Consulta integral de especialidades,CEX,N,N,NaN,...,78023791,950157989,NaN,NaN,los nogales,La Florida,78023791,950157989,NaN,NaN
1715865236101300582,DR. ALVARO ANDRES SANDOVAL CERDA,Rehabilitación: Prótesis Removible,29-05-2024,1030,Agendada,Derivación Odontologica,PRI,N,N,NaN,...,68249594,22828994,NaN,NaN,SEIS,La Florida,68249594,22828994,NaN,NaN
1715865236101300582,DR. ALVARO ANDRES SANDOVAL CERDA,Rehabilitación: Prótesis Removible,29-05-2024,1030,Agendada,Derivación Odontologica,PRI,N,N,NaN,...,68249594,22828994,NaN,NaN,SEIS,La Florida,68249594,22828994,NaN,NaN
1715270493452310369,DR. GABRIELA LUPE CALDERARA CEA,Otorrinolaringología,29-05-2024,840,Agendada,Derivacion desde APS,PRI,S,S,Hipoacusia Bilateral en Personas de 65 años y ...,...,34615263,24718849,NaN,NaN,rolando frodden,La Florida,34615263,24718849,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1713453317365300554,DR. LEONARDO JOSÉ SIRI ZUNINO,Infectología Adulto,19-12-2024,1040,Agendada,Control - Consulta integral de especialidades,CEX,N,N,NaN,...,967782271,951077542,NaN,NaN,VICUÑA MACKENNA,San Joaquín,967782271,951077542,NaN,NaN
1715610771664310021,DR. LEONARDO JOSÉ SIRI ZUNINO,Infectología Adulto,19-12-2024,1140,Agendada,Control - Consulta integral de especialidades,CEX,N,N,NaN,...,0937564768,NaN,NaN,NaN,serrano,La Florida,0937564768,NaN,NaN,NaN
1715610771664310021,DR. LEONARDO JOSÉ SIRI ZUNINO,Infectología Adulto,19-12-2024,1140,Agendada,Control - Consulta integral de especialidades,CEX,N,N,NaN,...,0937564768,NaN,NaN,NaN,serrano,La Florida,0937564768,NaN,NaN,NaN
1715270935281300910,DR. LEONARDO JOSÉ SIRI ZUNINO,Infectología Adulto,19-12-2024,1100,Agendada,Control - Consulta integral de especialidades,CEX,N,N,NaN,...,921712085,NaN,NaN,NaN,juan bagynka,La Florida,921712085,NaN,NaN,NaN


In [60]:
# Eliminar duplicados por índice y mantener el primero
df_sin_duplicados = dataframe_fu.drop_duplicates(keep='first')


In [61]:
df_sin_duplicados['ESTADO_CITA']

1713458226593310985                    Agendada
1715865236101300582                    Agendada
1715270493452310369                    Agendada
1715785618503310603                    Agendada
1710949111257900091    En buzón de reasignación
                                 ...           
1713451086016300967                    Agendada
1715609402723310398                    Agendada
1715270935281300910                    Agendada
1713453317365300554                    Agendada
1715610771664310021                    Agendada
Name: ESTADO_CITA, Length: 26086, dtype: object

In [62]:
df_sin_duplicados['ESTADO_CITA'].unique()

array(['Agendada', 'En buzón de reasignación', 'Anulada'], dtype=object)

In [63]:
df_sin_duplicados[df_sin_duplicados['ESPECIALIDAD_AGENDA']=='Imagenología AMB']['DESCRIPCION']

1715781854900300036                              Rx Pie Izq F-L
1715887712769300780                             Rx Mano Izq F-L
1715792560910300977                      TAC de Abdomen, con MC
1715792671879300991                       TAC de Pelvis, con MC
1716846144196310135                        TAC de Tórax, sin MC
                                         ...                   
1716406735799300462                              Rx Pie Izq F-L
1712323640596910122                    Rx Cadera Der Crosstable
1712323464883910069    Rx Pelvis AP para evaluación de Prótesis
1716830805648300587                                Rx Pelvis AP
1703877427182900912               TC de Abdomen y Pelvis con MC
Name: DESCRIPCION, Length: 1362, dtype: object

In [64]:
list(df_sin_duplicados)

['NOMBRE_AGENDA',
 'ESPECIALIDAD_AGENDA',
 'CITAFECH',
 'CITAHORA',
 'ESTADO_CITA',
 'DESCRIPCION',
 'ORIGEN',
 'GES_CITA',
 'GES_SOL',
 'PATOLOGIA_GES_SOL',
 'PACIENTE',
 'CITA_CODUTEN',
 'DOCUMENTO',
 'NOMBRE_TIPO_DOCUMENTO',
 'NUM_DOCUMENTO_TUTOR',
 'NOMBRE_DOCUMENTO_TUTOR',
 'CITAPACIFECHNACI',
 'SEXO',
 'NACIONALIDAD',
 'FACT_FINANCIADOR',
 'FACT_COBERTURA',
 'GES_EPISODIO',
 'MOVIL_HABITUAL',
 'TELEFONO1_HABITUAL',
 'TELEFONO2_HABITUAL',
 'TELEFONO3_HABITUALDIRECCION_HABITUAL',
 'COMUNA_HABITUAL',
 'MOVIL_CONTACTO',
 'TELEFONO1_CONTACTO',
 'TELEFONO2_CONTACTO',
 'TELEFONO3_CONTACTO',
 'CITA_KEY']

In [65]:
# Filtrar por las especialidades de Endocrinología Adulto y Endocrinología Infantil
endocrinologia_df_fu = df_sin_duplicados[df_sin_duplicados['ESPECIALIDAD_AGENDA'].isin(['Endocrinología Adulto','Cirugia Cabeza y Cuello','Imagenología AMB'])]
    # Filtrar las filas que no contienen 'No realizada' o 'Anulada' en la columna 'ESTADO_CITA'
endocrinologia_df_fu = endocrinologia_df_fu[~endocrinologia_df_fu['ESTADO_CITA'].isin(['Anulada', 'En buzón de reasignación'])]

In [66]:
list(endocrinologia_df_fu.columns)

['NOMBRE_AGENDA',
 'ESPECIALIDAD_AGENDA',
 'CITAFECH',
 'CITAHORA',
 'ESTADO_CITA',
 'DESCRIPCION',
 'ORIGEN',
 'GES_CITA',
 'GES_SOL',
 'PATOLOGIA_GES_SOL',
 'PACIENTE',
 'CITA_CODUTEN',
 'DOCUMENTO',
 'NOMBRE_TIPO_DOCUMENTO',
 'NUM_DOCUMENTO_TUTOR',
 'NOMBRE_DOCUMENTO_TUTOR',
 'CITAPACIFECHNACI',
 'SEXO',
 'NACIONALIDAD',
 'FACT_FINANCIADOR',
 'FACT_COBERTURA',
 'GES_EPISODIO',
 'MOVIL_HABITUAL',
 'TELEFONO1_HABITUAL',
 'TELEFONO2_HABITUAL',
 'TELEFONO3_HABITUALDIRECCION_HABITUAL',
 'COMUNA_HABITUAL',
 'MOVIL_CONTACTO',
 'TELEFONO1_CONTACTO',
 'TELEFONO2_CONTACTO',
 'TELEFONO3_CONTACTO',
 'CITA_KEY']

In [67]:
# Convertir la columna de fecha a formato datetime
#endocrinologia_df_fu['CITAFECH'] = pd.to_datetime(endocrinologia_df_fu['CITAFECH'])

# Formatear la columna de fecha en el formato deseado
#endocrinologia_df_fu['CITAFECH'] = endocrinologia_df_fu['CITAFECH'].dt.strftime('%Y-%m-%d')

In [68]:
endocrinologia_df_fu

,NOMBRE_AGENDA,ESPECIALIDAD_AGENDA,CITAFECH,CITAHORA,ESTADO_CITA,DESCRIPCION,ORIGEN,GES_CITA,GES_SOL,PATOLOGIA_GES_SOL,...,MOVIL_HABITUAL,TELEFONO1_HABITUAL,TELEFONO2_HABITUAL,TELEFONO3_HABITUALDIRECCION_HABITUAL,COMUNA_HABITUAL,MOVIL_CONTACTO,TELEFONO1_CONTACTO,TELEFONO2_CONTACTO,TELEFONO3_CONTACTO,CITA_KEY
1715781854900300036,Sala I-3,Imagenología AMB,29-05-2024,900,Agendada,Rx Pie Izq F-L,CEX,N,N,NaN,...,90382924,90382924,NaN,NaN,Tachira,La Florida,90382924,90382924,NaN,NaN
1715887712769300780,Sala I-3,Imagenología AMB,29-05-2024,910,Agendada,Rx Mano Izq F-L,CEX,N,N,NaN,...,90382924,90382924,NaN,NaN,Tachira,La Florida,90382924,90382924,NaN,NaN
1715792560910300977,TAC 2,Imagenología AMB,29-05-2024,830,Agendada,"TAC de Abdomen, con MC",CEX,N,N,NaN,...,77230332,23133687,NaN,NaN,Universidad de Chile,La Florida,77230332,23133687,NaN,NaN
1715792671879300991,TAC 2,Imagenología AMB,29-05-2024,830,Agendada,"TAC de Pelvis, con MC",CEX,N,N,NaN,...,77230332,23133687,NaN,NaN,Universidad de Chile,La Florida,77230332,23133687,NaN,NaN
1716846144196310135,TAC,Imagenología AMB,29-05-2024,815,Agendada,"TAC de Tórax, sin MC",CEX,N,N,NaN,...,30217621,0,NaN,NaN,pisa,La Florida,30217621,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1716406735799300462,Sala I-3,Imagenología AMB,21-08-2024,1400,Agendada,Rx Pie Izq F-L,CEX,N,N,NaN,...,96922799,978083464,977655435,NaN,SANTA MONICA,La Florida,96922799,978083464,977655435,NaN
1712323640596910122,Sala I-4,Imagenología AMB,04-10-2024,840,Agendada,Rx Cadera Der Crosstable,CEX,N,N,NaN,...,967396722,965981456,09965981456,NaN,SAN JOSE DE LA ESTRELLA,La Florida,967396722,965981456,09965981456,NaN
1712323464883910069,Sala I-4,Imagenología AMB,04-10-2024,830,Agendada,Rx Pelvis AP para evaluación de Prótesis,CEX,N,N,NaN,...,967396722,965981456,09965981456,NaN,SAN JOSE DE LA ESTRELLA,La Florida,967396722,965981456,09965981456,NaN
1716830805648300587,Sala I-3,Imagenología AMB,12-11-2024,1100,Agendada,Rx Pelvis AP,CEX,N,N,NaN,...,090000000000,NaN,NaN,NaN,NaN,La Florida,090000000000,NaN,NaN,NaN


In [69]:
endocrinologia_df_fu['DV'] = endocrinologia_df_fu['DOCUMENTO'].str[-1]

endocrinologia_df_fu['RUT'] = endocrinologia_df_fu['DOCUMENTO'].str[:-1]

In [70]:
endocrinologia_df_fu['NOMBRE_TIPO_DOCUMENTO']

1715781854900300036    RUN
1715887712769300780    RUN
1715792560910300977    RUN
1715792671879300991    RUN
1716846144196310135    RUN
                      ... 
1716406735799300462    RUN
1712323640596910122    RUN
1712323464883910069    RUN
1716830805648300587    RUN
1703877427182900912    RUN
Name: NOMBRE_TIPO_DOCUMENTO, Length: 1648, dtype: object

In [71]:
endocrinologia_df_fu = endocrinologia_df_fu[endocrinologia_df_fu['NOMBRE_TIPO_DOCUMENTO']=='RUN']

In [72]:
endocrinologia_df_fu['RUT'] = endocrinologia_df_fu['RUT'].astype(int)

In [73]:
endocrinologia_df_fu = endocrinologia_df_fu[['RUT','NOMBRE_AGENDA','ESPECIALIDAD_AGENDA','CITAFECH','CITAHORA','ESTADO_CITA','DESCRIPCION','ORIGEN','GES_CITA','GES_SOL','PATOLOGIA_GES_SOL','PACIENTE','CITA_CODUTEN','DOCUMENTO']]

In [74]:
endocrinologia_df_fu

,RUT,NOMBRE_AGENDA,ESPECIALIDAD_AGENDA,CITAFECH,CITAHORA,ESTADO_CITA,DESCRIPCION,ORIGEN,GES_CITA,GES_SOL,PATOLOGIA_GES_SOL,PACIENTE,CITA_CODUTEN,DOCUMENTO
1715781854900300036,4608028,Sala I-3,Imagenología AMB,29-05-2024,900,Agendada,Rx Pie Izq F-L,CEX,N,N,NaN,SILVIA VENEGAS ARCE,AC00018782,46080289
1715887712769300780,4608028,Sala I-3,Imagenología AMB,29-05-2024,910,Agendada,Rx Mano Izq F-L,CEX,N,N,NaN,SILVIA VENEGAS ARCE,AC00018782,46080289
1715792560910300977,4653276,TAC 2,Imagenología AMB,29-05-2024,830,Agendada,"TAC de Abdomen, con MC",CEX,N,N,NaN,OCTAVIO DEL CARMEN SALGADO BUSTAMANTE,AC00019110,46532767
1715792671879300991,4653276,TAC 2,Imagenología AMB,29-05-2024,830,Agendada,"TAC de Pelvis, con MC",CEX,N,N,NaN,OCTAVIO DEL CARMEN SALGADO BUSTAMANTE,AC00019110,46532767
1716846144196310135,5209104,TAC,Imagenología AMB,29-05-2024,815,Agendada,"TAC de Tórax, sin MC",CEX,N,N,NaN,MARÍA TERESA PALMA CUBILLOS,AC00025538,52091047
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1716406735799300462,5636179,Sala I-3,Imagenología AMB,21-08-2024,1400,Agendada,Rx Pie Izq F-L,CEX,N,N,NaN,HERNÁN PATRICIO GONZÁLEZ NÚÑEZ,AC00427431,56361790
1712323640596910122,22889659,Sala I-4,Imagenología AMB,04-10-2024,840,Agendada,Rx Cadera Der Crosstable,CEX,N,N,NaN,MATÍAS ANTONIO HUMERES MADARIAGA,AC00387114,228896594
1712323464883910069,22889659,Sala I-4,Imagenología AMB,04-10-2024,830,Agendada,Rx Pelvis AP para evaluación de Prótesis,CEX,N,N,NaN,MATÍAS ANTONIO HUMERES MADARIAGA,AC00387114,228896594
1716830805648300587,28119064,Sala I-3,Imagenología AMB,12-11-2024,1100,Agendada,Rx Pelvis AP,CEX,N,N,NaN,BENJAMIN EXEQUIEL JARA CLAVERIA,AC00519708,281190644


In [75]:
# Día actual
today = date.today()

# Fecha actual
now = datetime.now()

# Formatear día con ceros a la izquierda si es necesario
day_formatted = "{:02d}".format(today.day)


In [76]:
wb = Workbook()
ws = wb.active

for r in dataframe_to_rows(endocrinologia_df_fu, index= False, header = True):
    ws.append(r)

for cell in ws['D'][1:]:
    cell.number_format = 'DD-MM-YYYY'
nombre_archivo = f'{today.year}{month_formatted}{day_formatted}_citas_futuras_tiroides.xlsx'
wb.save(os.path.join(dir_actual, '..','CA TIROIDES',f'{nombre_archivo}'))